# Analysis

This notebook contains the code necessary to conduct all data analyses for the *Pan* 3d Genome project. I have organized this largely by order of appearance in the manuscript; however, a few sections may be out of order. Use the Table of Contents below to navigate to specific analyses.

## Table of Contents

- [Notebook Setup](#notebooksetup)
- [Data Description](#datadescription)
    - [N Windows and Comparisons](#nwindowscomparisons)
    - [Divergence Score Distribution](#divergencescoredistribution)
- [*Pan*-*Homo* Divergence Distribution Comparison](#panhomodivergence)
- [Minimally Divergent Windows and Ultraconserved TAD Boundaries](#okhovatetal2023)
- [Lineage Comparison](#lineagecomparison)
- [Hierarchical Clustering](#hierachicalclustering)
    - [Establish N Clusters, Cluster Size, and Cluster Composition](#establishclusters)
    - [Single Divergent Individual Clustering Windows](#singledivergentindividualclusteringwindows)
    - [Lineage-Specific Clustering Windows](#lineagespecificclusteringwindows)
    - [Multiple Divergent Individuals Clustering Windows](#multipledivergentindividualsclusteringwindows)
    - [Chr21 Window Topologies](#chr21windowtopologies)
    - [Window Topology Divergence Differences](#windowtopologydivergencedifferences)
    - [Three, Four, and Five Cluster Window Topologies](#threefourfiveclusterwindowtopologies)
- [Bonobo-Chimpanzee Windows](#bonobochimpanzeewindows)
    - [Genes Total and Genes per Window](#genestotalgenesperwindow)
    - [phastCons Elements](#phastconselements)
- [Phenotype Enrichment](#phenotypeenrichment)
- [*In Silico* Mutagenesis for Bonobo-Specific Variants](#ppninsilicomutagenesis)
    - [Ancestral Allele](#ancestralallele)
    - [Explained Divergence](#explaineddivergence)
    - [CTCF](#ctcf)
    - [Chromatin Contact Effect](#chromatincontacteffect)
- [*In Silico* Mutagenesis for Chimpanzee Lineage-Specific Variants](#ptrinsilicomutagenesis)
- [Bonobo-Chimpanzee Gene Expression Differences](#geneexpression)
- [Cell Type Comparison](#celltypecomparison)

## Notebook Setup <a class = 'anchor' id = 'notebooksetup'></a>

Load all needed packages, change directories, and load the HFF comparisons dataframe that we previously generated in the last notebook.

In [1]:
import json
import numpy as np
import pandas as pd
import pybedtools

from scipy.stats import fisher_exact
from scipy.stats import kstest
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr

pd.options.display.max_columns = 100
pd.options.display.max_rows = 500

In [2]:
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist

In [3]:
import matplotlib.font_manager as font_manager
arial_path = '/wynton/home/capra/cbrand/miniconda3/envs/jupyter/fonts/Arial.ttf'
arial = font_manager.FontProperties(fname = arial_path)

In [4]:
cd /wynton/group/capra/projects/pan_3d_genome/data

/wynton/group/capra/projects/pan_3d_genome/data


In [5]:
comparisons = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/dataframes/HFF_comparisons.txt', sep = '\t', header = 0)
comparisons.head(5)

,ind1,ind2,lineages,chr,window_start,window,mse,spearman,divergence,seq_diff
0,Akwaya-Jean,Alfred,pte-ptt,chr10,1572864,chr10_1572864,0.000168,0.987655,0.012345,2803
1,Akwaya-Jean,Alfred,pte-ptt,chr10,2097152,chr10_2097152,0.000481,0.969809,0.030191,2715
2,Akwaya-Jean,Alfred,pte-ptt,chr10,2621440,chr10_2621440,0.001675,0.996398,0.003602,2849
3,Akwaya-Jean,Alfred,pte-ptt,chr10,3145728,chr10_3145728,0.000323,0.997899,0.002101,2606
4,Akwaya-Jean,Alfred,pte-ptt,chr10,3670016,chr10_3670016,0.000143,0.996732,0.003268,2594


Double check that the frame is the size that it should be.

In [6]:
len(comparisons)

6669390

## Data Description <a class = 'anchor' id = 'datadescription'></a>

### N Windows and Comparisons <a class = 'anchor' id = 'nwindowscomparisons'></a>

Let's generate some basic statistics for this dataset before diving into specific analyses. First, let's confirm the total number of windows and the number of windows per chromosome.

In [7]:
len(comparisons['window'].unique())

4420

In [8]:
windows_df = pd.DataFrame(comparisons['window'].unique(), columns = ['window'])
windows_df = windows_df['window'].str.split('_', expand = True).rename(columns = {0:'chr', 1:'window_start'})
windows_df.groupby(['chr']).size().to_frame('N')

,N
chr,
chr1,341
chr10,208
chr11,226
chr12,210
chr13,166
chr14,151
chr15,120
chr16,102
chr17,99


How many comparisons are there per autosomal window? X chromosome window?

In [9]:
len(comparisons[comparisons['window'] == 'chr1_1048576'])

1540

In [10]:
len(comparisons[comparisons['window'] == 'chrX_4718592'])

630

How many comparisons are there per chromosome?

In [11]:
comparisons.groupby('chr').size().to_frame('N')

,N
chr,
chr1,525140
chr10,320320
chr11,348040
chr12,323400
chr13,255640
chr14,232540
chr15,184800
chr16,157080
chr17,152460


What is the distribution of lineage comparisons for a single autosomal window?

In [12]:
comparisons[comparisons['window'] == 'chr1_1048576'].groupby('lineages').size().to_frame('N').sort_values(by = 'N', ascending = False)

,N
lineages,
ppn-ptr,423
pts-ptt,272
pts-ptv,153
ptt-ptv,144
pts-pts,136
ptt-ptt,120
pte-pts,85
pte-ptt,80
pte-ptv,45


For a single chrX window?

In [13]:
comparisons[comparisons['window'] == 'chrX_4718592'].groupby('lineages').size().to_frame('N').sort_values(by = 'N', ascending = False)

,N
lineages,
ppn-ptr,203
pts-ptt,121
pts-pts,55
pts-ptv,55
ptt-ptt,55
ptt-ptv,55
pte-pts,22
pte-ptt,22
ppn-ppn,21


### Divergence Score Distribution <a class = 'anchor' id = 'divergencescoredistribution'></a>

What does the distribution of divergence scores look like?

In [14]:
comparisons['divergence'].min()

2.492599999737166e-07

In [15]:
comparisons['divergence'].max()

0.873604563894

How many comparisons have a divergence score < 0.01?

In [16]:
len(comparisons[comparisons['divergence'] < 0.01])

5539567

In [17]:
5539567/6669390

0.830595751635457

What is the maximum divergence score per window?

In [18]:
maxes = comparisons.groupby(['window'])['divergence'].max().to_frame('max').reset_index()
maxes

,window,max
0,chr10_100139008,0.128959
1,chr10_100663296,0.101497
2,chr10_101187584,0.033608
3,chr10_101711872,0.138848
4,chr10_102236160,0.031867
...,...,...
4415,chrX_94371840,0.076931
4416,chrX_94896128,0.044540
4417,chrX_95420416,0.002984
4418,chrX_99090432,0.030925


How many windows have a maximum divergence of 0.005 and 0.01? What is the maximum divergence for various quantiles?

In [19]:
len(maxes[maxes['max'] < 0.005])

245

In [20]:
len(maxes[maxes['max'] < 0.01])

874

In [21]:
maxes['max'].quantile(q = 0.1)

0.006717990286899901

In [22]:
maxes['max'].quantile(q = 0.25)

0.0119926032

In [23]:
maxes['max'].quantile(q = 0.5)

0.02476536453449995

In [24]:
maxes['max'].quantile(q = 0.9)

0.11610571464919983

Get divergence scores for Figure 1C examples.

In [25]:
comparisons[(comparisons['window'] == 'chr4_77594624') & (comparisons['ind1'] == 'Maya') & (comparisons['ind2'] == 'Washu')]

,ind1,ind2,lineages,chr,window_start,window,mse,spearman,divergence,seq_diff
6211267,Maya,Washu,pts-pts,chr4,77594624,chr4_77594624,1.090777e-07,1.0,4.526210e-07,11


In [26]:
comparisons[(comparisons['window'] == 'chr18_46137344') & (comparisons['ind1'] == 'Julie-A959') & (comparisons['ind2'] == 'Vincent')]

,ind1,ind2,lineages,chr,window_start,window,mse,spearman,divergence,seq_diff
5136420,Julie-A959,Vincent,pts-ptt,chr18,46137344,chr18_46137344,0.020585,0.350611,0.649389,3049


## *Pan* - *Homo* Divergence Distribution Comparison <a class = 'anchor' id = 'panhomodivergence'></a>

Compare the *Pan* divergence score distribution to the distribution calculated from 130 modern human genomes.

In [27]:
human_comparisons = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/comparisons/thousand_genomes_subset_HFF/melted_130v130_1KG_subsample.csv', sep = ',', header = 0)
human_comparisons.head(5)

,ind1,ind2,chrm,start_pos,3d_divergence
0,AFR_ASW_female_NA19917,AFR_ASW_female_NA19901,chr1,1048576,0.002916
1,AFR_ASW_female_NA19917,AFR_ASW_female_NA20314,chr1,1048576,0.003368
2,AFR_ASW_female_NA19917,AFR_ASW_female_NA20317,chr1,1048576,0.005616
3,AFR_ASW_female_NA19917,AFR_ASW_female_NA19625,chr1,1048576,0.001083
4,AFR_ASW_female_NA19917,AFR_ACB_female_HG02337,chr1,1048576,0.004772


In [28]:
len(human_comparisons)

40860105

In [29]:
pan_comparisons = comparisons[comparisons['chr'] != 'chrX']

In [30]:
len(pan_comparisons)

6574260

In [31]:
pan_comparisons['divergence'].mean()

0.007906218997677883

In [32]:
human_comparisons['3d_divergence'].mean()

0.003348655193871765

In [33]:
kstest(pan_comparisons['divergence'], human_comparisons['3d_divergence'])

KstestResult(statistic=0.3294594800502246, pvalue=0.0)

## Minimally Divergent Windows and Ultraconserved TAD Boundaries <a class = 'anchor' id = 'okhovatetal2023'></a>

Let's consider the overlap between minimally divergent windows and ultra- and primate-conserved boundaries from Okhovat et al. 2023. First, let's read in the files containing the boundaries in panTro6 coordinates.

In [34]:
ultraconserved_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/Okhovat_et_al_2023/Okhovat_et_al_2023_ultraconserved_boundaries_panTro6.bed')
ultraconserved_pbtBED.head(5)

chr1	1098611	1204332	hg38_n_B00132
 chr1	2029035	2038921	hg38_n_B00190
 chr1	5402415	5412399	hg38_n_B00510
 chr1	7054986	7064987	hg38_n_B00676
 chr1	7946002	7956016	hg38_n_B00765
 

In [35]:
len(ultraconserved_pbtBED)

984

In [36]:
primate_conserved_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/Okhovat_et_al_2023/Okhovat_et_al_2023_primate_conserved_boundaries_panTro6.bed')
primate_conserved_pbtBED.head(5)

chr1	13254423	13264412	hg38_n_B01247
 chr1	13745195	13755219	hg38_n_B01296
 chr1	17375951	17385984	hg38_n_B01639
 chr1	24694359	24704375	hg38_n_B02283
 chr1	25957982	25967966	hg38_n_B02394
 

In [37]:
len(primate_conserved_pbtBED)

459

Do these sets overlap at all?

In [38]:
ultraconserved_primate_conserved_intersect = ultraconserved_pbtBED.intersect(primate_conserved_pbtBED)

In [39]:
len(ultraconserved_primate_conserved_intersect)

0

Intersect the TAD boundaries with the available panTro6 windows to determine the maximum amount of overlap.

In [40]:
windows_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/metadata/panTro6_windows_with_full_coverage.bed')
windows_pbtBED.head(5)

chr1	1048576	2097152
 chr1	1572864	2621440
 chr1	2097152	3145728
 chr1	2621440	3670016
 chr1	3145728	4194304
 

In [41]:
len(ultraconserved_pbtBED.intersect(windows_pbtBED, u = True))

915

In [42]:
len(primate_conserved_pbtBED.intersect(windows_pbtBED, u = True))

415

What is the *Pan* divergence distribution for both sets of regions? Start by creating a pybedtools object for the maxes dataframe.

In [43]:
window_max_divergence = maxes['max']
maxes_BED = maxes['window'].str.split('_', expand = True).rename(columns = {0:'chr', 1:'start'})
maxes_BED['start'] = maxes_BED['start'].astype(int)
maxes_BED['end'] = maxes_BED['start'] + 1048576
maxes_BED['max'] = window_max_divergence
maxes_BED.head(5)

,chr,start,end,max
0,chr10,100139008,101187584,0.128959
1,chr10,100663296,101711872,0.101497
2,chr10,101187584,102236160,0.033608
3,chr10,101711872,102760448,0.138848
4,chr10,102236160,103284736,0.031867


In [44]:
len(maxes_BED)

4420

In [45]:
maxes_pbtBED = pybedtools.BedTool().from_dataframe(maxes_BED).sort()
maxes_pbtBED.head(5)

chr1	1048576	2097152	0.036526326318
 chr1	1572864	2621440	0.006578744897
 chr1	2097152	3145728	0.015877001035
 chr1	2621440	3670016	0.079123467404
 chr1	3145728	4194304	0.192662189319
 

Now intersect with the ultraconserved regions. Some TAD boundaries may overlie two overlapping windows. We will compute a centrality score and use the maximum divergence from the window in which the TAD boundary is most central.

The centrality score below is computed as:

$$ \mathrm{Centrality\:score} = |0.5 - \frac{(\frac{\mathrm{TAD\:Boundary\:End\:-\:TAD\:Boundary\:Start}}{2} + \mathrm{TAD\:Boundary\:Start}) - \mathrm{Window\:Start}}{1048576}|$$ 

Scores at or near 0 indicate the TAD boundary is more central to that window where values approaching 0.5 are at the window's edge.

Run the intersection for the ultraconserved.

In [46]:
ultraconserved_maxes_intersect = ultraconserved_pbtBED.intersect(maxes_pbtBED, wa = True, wb = True).to_dataframe(names=['tb_chr','tb_start','tb_end','tb_id','window_chr','window_start','window_end','max'])
ultraconserved_maxes_intersect['centrality_score'] = abs(0.5 - (((((ultraconserved_maxes_intersect['tb_end'] - ultraconserved_maxes_intersect['tb_start']) / 2) + ultraconserved_maxes_intersect['tb_start']) - ultraconserved_maxes_intersect['window_start']) / 1048576))  # add a center score to assess centrality of TAD boundary when one overlaps multiple 3d windows
ultraconserved_maxes_intersect = ultraconserved_maxes_intersect.sort_values(by=['tb_chr','tb_start','centrality_score'], ascending = True)
ultraconserved_maxes_intersect.head(5)

,tb_chr,tb_start,tb_end,tb_id,window_chr,window_start,window_end,max,centrality_score
0,chr1,1098611,1204332,hg38_n_B00132,chr1,1048576,2097152,0.036526,0.401871
2,chr1,2029035,2038921,hg38_n_B00190,chr1,1572864,2621440,0.006579,0.060247
1,chr1,2029035,2038921,hg38_n_B00190,chr1,1048576,2097152,0.036526,0.439753
3,chr1,5402415,5412399,hg38_n_B00510,chr1,5242880,6291456,0.017641,0.343095
4,chr1,7054986,7064987,hg38_n_B00676,chr1,6291456,7340032,0.002978,0.232928


In [47]:
ultraconserved_maxes_intersect = ultraconserved_maxes_intersect.drop_duplicates(subset = ['tb_chr','tb_start','tb_end'], keep = 'first')
ultraconserved_maxes_intersect.head(5)

,tb_chr,tb_start,tb_end,tb_id,window_chr,window_start,window_end,max,centrality_score
0,chr1,1098611,1204332,hg38_n_B00132,chr1,1048576,2097152,0.036526,0.401871
2,chr1,2029035,2038921,hg38_n_B00190,chr1,1572864,2621440,0.006579,0.060247
3,chr1,5402415,5412399,hg38_n_B00510,chr1,5242880,6291456,0.017641,0.343095
4,chr1,7054986,7064987,hg38_n_B00676,chr1,6291456,7340032,0.002978,0.232928
6,chr1,7946002,7956016,hg38_n_B00765,chr1,7340032,8388608,0.021275,0.082673


In [48]:
len(ultraconserved_maxes_intersect)

915

In [49]:
ultraconserved_maxes_intersect['max'].to_csv('/wynton/group/capra/projects/pan_3d_genome/data/Okhovat_et_al_2023/ultraconserved_window_maxes.txt', sep = '\t', header = False, index = False)

Now the primate-conserved.

In [50]:
primate_conserved_maxes_intersect = primate_conserved_pbtBED.intersect(maxes_pbtBED, wa = True, wb = True).to_dataframe(names=['tb_chr','tb_start','tb_end','tb_id','window_chr','window_start','window_end','max'])
primate_conserved_maxes_intersect['centrality_score'] = abs(0.5 - (((((primate_conserved_maxes_intersect['tb_end'] - primate_conserved_maxes_intersect['tb_start']) / 2) + primate_conserved_maxes_intersect['tb_start']) - primate_conserved_maxes_intersect['window_start']) / 1048576))  # add a center score to assess centrality of TAD boundary when one overlaps multiple 3d windows
primate_conserved_maxes_intersect = primate_conserved_maxes_intersect.sort_values(by=['tb_chr','tb_start','centrality_score'], ascending = True)
primate_conserved_maxes_intersect = primate_conserved_maxes_intersect.drop_duplicates(subset = ['tb_chr','tb_start','tb_end'], keep = 'first')
primate_conserved_maxes_intersect.head(5)

,tb_chr,tb_start,tb_end,tb_id,window_chr,window_start,window_end,max,centrality_score
0,chr1,13254423,13264412,hg38_n_B01247,chr1,12582912,13631488,0.140253,0.145166
3,chr1,13745195,13755219,hg38_n_B01296,chr1,13107200,14155776,0.039724,0.113219
4,chr1,17375951,17385984,hg38_n_B01639,chr1,16777216,17825792,0.028705,0.075782
6,chr1,24694359,24704375,hg38_n_B02283,chr1,24117248,25165824,0.015984,0.055152
9,chr1,25957982,25967966,hg38_n_B02394,chr1,25690112,26738688,0.009441,0.239779


In [51]:
len(primate_conserved_maxes_intersect)

415

In [52]:
primate_conserved_maxes_intersect['max'].to_csv('/wynton/group/capra/projects/pan_3d_genome/data/Okhovat_et_al_2023/primate_conserved_window_maxes.txt', sep = '\t', header = False, index = False)

Are these distributions different? Run a KS test. First the ultraconserved.

In [53]:
kstest(maxes['max'], ultraconserved_maxes_intersect['max'])

KstestResult(statistic=0.18024379991593106, pvalue=5.261687464016495e-22)

In [54]:
maxes['max'].mean()

0.05016095426516081

In [55]:
ultraconserved_maxes_intersect['max'].mean()

0.02460737114364039

Now the primate-conserved.

In [56]:
kstest(maxes['max'], primate_conserved_maxes_intersect['max'])

KstestResult(statistic=0.12690127023932835, pvalue=8.713086838339734e-06)

In [57]:
primate_conserved_maxes_intersect['max'].mean()

0.03320046814213971

Export the window maxes as a genome-wide background for visualization.

In [58]:
maxes['max'].to_csv('/wynton/group/capra/projects/pan_3d_genome/data/Okhovat_et_al_2023/window_maxes.txt', sep = '\t', header = False, index = False)

## Lineage Comparison <a class = 'anchor' id = 'lineagecomparison'></a>

Let's look at 3d divergence by lineages represented in the comparison. We will simplify things by collapsing when chimpanzees of different subspecies are compared.

In [59]:
simple_dyad_comparisons = comparisons[['ind1','ind2','lineages','divergence']].copy()
simple_dyad_comparisons['lineages'] = simple_dyad_comparisons['lineages'].replace({'pte-pts':'pt-pt', 'pte-ptt':'pt-pt', 'pte-ptv':'pt-pt', 'pts-ptt':'pt-pt', 'pts-ptv':'pt-pt', 'ptt-ptv':'pt-pt'})
simple_dyad_comparisons.head(5)

,ind1,ind2,lineages,divergence
0,Akwaya-Jean,Alfred,pt-pt,0.012345
1,Akwaya-Jean,Alfred,pt-pt,0.030191
2,Akwaya-Jean,Alfred,pt-pt,0.003602
3,Akwaya-Jean,Alfred,pt-pt,0.002101
4,Akwaya-Jean,Alfred,pt-pt,0.003268


In [60]:
simple_dyad_comparisons.groupby('lineages')['divergence'].median().to_frame('median')

,median
lineages,
ppn-ppn,0.000829
ppn-ptr,0.004130
pt-pt,0.002331
pte-pte,0.001282
pts-pts,0.001629
ptt-ptt,0.002165
ptv-ptv,0.000590


## Hierarchical Clustering <a class = 'anchor' id = 'hierachicalclustering'></a>

### Establish N Clusters, Cluster Size, and Cluster Composition <a class = 'anchor' id = 'establishclusters'></a>

Run a script to generate trees per window, grab the cluster identity per individual, and catch the y-coordinates of the top four nodes in the tree. 

In [61]:
complete_linkage_trees = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/complete_linkage_clustering_per_window.txt', sep = '\t', header = None)

In [62]:
autosomal_complete_linkage_trees = complete_linkage_trees[~complete_linkage_trees[0].str.startswith('chrX_')]
chrX_complete_linkage_trees = complete_linkage_trees[complete_linkage_trees[0].str.startswith('chrX_')]

In [63]:
autosomal_trees_header = ['window','Akwaya-Jean','Alfred','Alice','Andromeda','Athanga','Berta','Bihati','Blanquita','Bono','Bosco','Brigitta','Bwamble','Cindy-schwein','Cindy-troglodytes','Cindy-verus','Cleo','Clint','Coco-chimp','Damian','Desmond','Doris','Dzeeta','Frederike','Gamin','Hermien','Hortense','Ikuru','Jimmie','Julie-A959','Julie-LWC21','Kidongo','Koby','Kombote','Kosana','Koto','Kumbuka','Lara','Linda','Luky','Marlin','Maya','Mgbadolite','Mirinda','Nakuu','Natalie','Negrita','SeppToni','Taweh','Tibe','Tongo','Trixie','Ula','Vaillant','Vincent','Washu','Yogui','node_0','node_1','node_2','node_3']
chrX_trees_header = ['window','Alice','Andromeda','Berta','Bihati','Blanquita','Cindy-schwein','Cindy-troglodytes','Cindy-verus','Cleo','Coco-chimp','Doris','Dzeeta','Frederike','Hermien','Hortense','Ikuru','Jimmie','Julie-A959','Julie-LWC21','Kidongo','Kombote','Kosana','Kumbuka','Lara','Linda','Luky','Marlin','Maya','Mirinda','Nakuu','Natalie','Negrita','Taweh','Tibe','Trixie','Ula','node_0','node_1','node_2','node_3']

autosomal_complete_linkage_trees.columns = autosomal_trees_header
chrX_complete_linkage_trees = chrX_complete_linkage_trees.dropna(axis = 1)
chrX_complete_linkage_trees.columns = chrX_trees_header

In [64]:
autosomal_complete_linkage_trees.head(5)

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3
0,chr10_1572864,C3,C3,C3,C2,C2,C3,C3,C2,C1,C3,C3,C2,C2,C3,C3,C2,C3,C2,C3,C1,C2,C1,C3,C3,C1,C1,C3,C3,C3,C3,C2,C3,C1,C1,C3,C1,C2,C3,C3,C3,C2,C2,C2,C2,C1,C2,C3,C3,C3,C2,C2,C3,C3,C2,C2,C2,0.014522,0.091748,0.091748,0.067514
1,chr10_2097152,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.038838,0.269334,0.269334,0.115208
2,chr10_2621440,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005674,0.012824,0.012824,0.006737
3,chr10_3145728,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002904,0.010145,0.010145,0.006418
4,chr10_3670016,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005140,0.096248,0.096248,0.048080


In [65]:
chrX_complete_linkage_trees.head(5)

,window,Alice,Andromeda,Berta,Bihati,Blanquita,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Coco-chimp,Doris,Dzeeta,Frederike,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Kombote,Kosana,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mirinda,Nakuu,Natalie,Negrita,Taweh,Tibe,Trixie,Ula,node_0,node_1,node_2,node_3
4269,chrX_4718592,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.000386870517,0.0942329238279999,0.0942329238279999,0.0244069347169999
4270,chrX_5242880,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C0,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.0,0.018755481251,0.018755481251,0.0144252191229999
4271,chrX_6815744,C1,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C1,C3,C3,C3,C3,C2,C3,C3,C1,C3,C3,C3,C3,C3,C2,C3,C3,C3,C3,C3,0.0004348450219999,0.0176649495409999,0.0176649495409999,0.014919062537
4272,chrX_9437184,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.0653681878299999,0.0653681878299999,0.020195340661
4273,chrX_11010048,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,7.667215399997929e-05,0.063108737967,0.063108737967,0.0414393167169999


Write a function that creates a cluster composition string. We will sort clusters by increasing size and delineate them by a '/'.

In [66]:
def generate_autosomal_clusters_lists(dataframe, window):
    row_index = dataframe[dataframe['window'] == window].index
    row = dataframe.loc[row_index,['Akwaya-Jean','Alfred','Alice','Andromeda','Athanga','Berta','Bihati','Blanquita','Bono','Bosco','Brigitta','Bwamble','Cindy-schwein','Cindy-troglodytes','Cindy-verus','Cleo','Clint','Coco-chimp','Damian','Desmond','Doris','Dzeeta','Frederike','Gamin','Hermien','Hortense','Ikuru','Jimmie','Julie-A959','Julie-LWC21','Kidongo','Koby','Kombote','Kosana','Koto','Kumbuka','Lara','Linda','Luky','Marlin','Maya','Mgbadolite','Mirinda','Nakuu','Natalie','Negrita','SeppToni','Taweh','Tibe','Tongo','Trixie','Ula','Vaillant','Vincent','Washu','Yogui']].values[0].tolist()
    cluster_values = sorted(list(set(row)))
    clusters_list = []
    single_clusters = 0
    
    for c in cluster_values:
        cluster_samples = dataframe.loc[row_index].apply(lambda row: row[row == c].index, axis = 1).values[0].tolist()
        if len(cluster_samples) == 1:
            single_clusters += 1
        cluster_samples = ' '.join(cluster_samples)
        clusters_list.append(cluster_samples)

    def count_spaces(s):
        return s.count(' ')

    sorted_data = sorted(clusters_list, key = count_spaces)
    cluster_composition = '/'.join(sorted_data)
    return len(clusters_list), single_clusters, cluster_composition

In [67]:
def generate_chrX_clusters_lists(dataframe, window):
    row_index = dataframe[dataframe['window'] == window].index
    row = dataframe.loc[row_index,['Alice','Andromeda','Berta','Bihati','Blanquita','Cindy-schwein','Cindy-troglodytes','Cindy-verus','Cleo','Coco-chimp','Doris','Dzeeta','Frederike','Hermien','Hortense','Ikuru','Jimmie','Julie-A959','Julie-LWC21','Kidongo','Kombote','Kosana','Kumbuka','Lara','Linda','Luky','Marlin','Maya','Mirinda','Nakuu','Natalie','Negrita','Taweh','Tibe','Trixie','Ula']].values[0].tolist()
    cluster_values = sorted(list(set(row)))
    clusters_list = []
    single_clusters = 0
    
    for c in cluster_values:
        cluster_samples = dataframe.loc[row_index].apply(lambda row: row[row == c].index, axis = 1).values[0].tolist()
        if len(cluster_samples) == 1:
            single_clusters += 1
        cluster_samples = ' '.join(cluster_samples)
        clusters_list.append(cluster_samples)
    
    def count_spaces(s):
        return s.count(' ')

    sorted_data = sorted(clusters_list, key = count_spaces)
    cluster_composition = '/'.join(sorted_data)
    return len(clusters_list), single_clusters, cluster_composition

In [68]:
autosomal_complete_linkage_trees[['n_clusters','n_single_clusters','cluster_composition']] = pd.DataFrame(autosomal_complete_linkage_trees.apply(lambda row: generate_autosomal_clusters_lists(autosomal_complete_linkage_trees, row['window']), axis=1).tolist(), index=autosomal_complete_linkage_trees.index)

/scratch/cbrand/ipykernel_2007303/587978040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  autosomal_complete_linkage_trees[['n_clusters','n_single_clusters','cluster_composition']] = pd.DataFrame(autosomal_complete_linkage_trees.apply(lambda row: generate_autosomal_clusters_lists(autosomal_complete_linkage_trees, row['window']), axis=1).tolist(), index=autosomal_complete_linkage_trees.index)
/scratch/cbrand/ipykernel_2007303/587978040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  autosomal_compl

In [69]:
autosomal_complete_linkage_trees.head(5)

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition
0,chr10_1572864,C3,C3,C3,C2,C2,C3,C3,C2,C1,C3,C3,C2,C2,C3,C3,C2,C3,C2,C3,C1,C2,C1,C3,C3,C1,C1,C3,C3,C3,C3,C2,C3,C1,C1,C3,C1,C2,C3,C3,C3,C2,C2,C2,C2,C1,C2,C3,C3,C3,C2,C2,C3,C3,C2,C2,C2,0.014522,0.091748,0.091748,0.067514,3,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1,chr10_2097152,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.038838,0.269334,0.269334,0.115208,2,0,Cindy-troglodytes Doris Luky Marlin/Akwaya-Jea...
2,chr10_2621440,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005674,0.012824,0.012824,0.006737,2,0,Cindy-troglodytes Doris Luky Marlin/Akwaya-Jea...
3,chr10_3145728,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002904,0.010145,0.010145,0.006418,2,0,Alice Cindy-verus Luky/Akwaya-Jean Alfred Andr...
4,chr10_3670016,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005140,0.096248,0.096248,0.048080,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...


In [70]:
chrX_complete_linkage_trees[['n_clusters','n_single_clusters','cluster_composition']]= pd.DataFrame(chrX_complete_linkage_trees.apply(lambda row: generate_chrX_clusters_lists(chrX_complete_linkage_trees, row['window']), axis=1).tolist(), index=chrX_complete_linkage_trees.index)

In [71]:
chrX_complete_linkage_trees.head(5)

,window,Alice,Andromeda,Berta,Bihati,Blanquita,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Coco-chimp,Doris,Dzeeta,Frederike,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Kombote,Kosana,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mirinda,Nakuu,Natalie,Negrita,Taweh,Tibe,Trixie,Ula,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition
4269,chrX_4718592,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.000386870517,0.0942329238279999,0.0942329238279999,0.0244069347169999,2,0,Alice Berta/Andromeda Bihati Blanquita Cindy-s...
4270,chrX_5242880,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C0,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.0,0.018755481251,0.018755481251,0.0144252191229999,3,1,Julie-A959/Doris Luky/Alice Andromeda Berta Bi...
4271,chrX_6815744,C1,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C3,C1,C3,C3,C3,C3,C2,C3,C3,C1,C3,C3,C3,C3,C3,C2,C3,C3,C3,C3,C3,0.0004348450219999,0.0176649495409999,0.0176649495409999,0.014919062537,3,0,Kosana Natalie/Alice Jimmie Linda/Andromeda Be...
4272,chrX_9437184,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.0653681878299999,0.0653681878299999,0.020195340661,2,1,Luky/Alice Andromeda Berta Bihati Blanquita Ci...
4273,chrX_11010048,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,7.667215399997929e-05,0.063108737967,0.063108737967,0.0414393167169999,2,0,Andromeda Maya/Alice Berta Bihati Blanquita Ci...


Concat the dataframes.

In [72]:
complete_linkage_trees = pd.concat([autosomal_complete_linkage_trees, chrX_complete_linkage_trees])

In [73]:
len(complete_linkage_trees)

4420

Take a look at the number of clusters.

In [74]:
complete_linkage_trees.groupby(['n_clusters'])['window'].count().to_frame('N')

,N
n_clusters,
2,3622
3,748
4,46
5,4


Let's get a table of unique cluster compositions for each set.

In [75]:
autosomal_complete_summary = autosomal_complete_linkage_trees.groupby(['cluster_composition'])['window'].count().to_frame('N')
chrX_complete_summary = chrX_complete_linkage_trees.groupby(['cluster_composition'])['window'].count().to_frame('N')

In [76]:
len(autosomal_complete_summary)

2745

In [77]:
len(chrX_complete_summary)

104

In [78]:
autosomal_complete_summary.sort_values(by = 'N', ascending = False).head(5)

,N
cluster_composition,
Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Doris Frederike Gamin Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Koto Lara Linda Luky Marlin Maya Mgbadolite Mirinda Nakuu Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui,315
Luky/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bono Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Desmond Doris Dzeeta Frederike Gamin Hermien Hortense Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Kombote Kosana Koto Kumbuka Lara Linda Marlin Maya Mgbadolite Mirinda Nakuu Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui,33
Desmond/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bono Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Doris Dzeeta Frederike Gamin Hermien Hortense Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Kombote Kosana Koto Kumbuka Lara Linda Luky Marlin Maya Mgbadolite Mirinda Nakuu Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui,30
Doris/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bono Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Desmond Dzeeta Frederike Gamin Hermien Hortense Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Kombote Kosana Koto Kumbuka Lara Linda Luky Marlin Maya Mgbadolite Mirinda Nakuu Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui,29
Marlin/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bono Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Desmond Doris Dzeeta Frederike Gamin Hermien Hortense Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Kombote Kosana Koto Kumbuka Lara Linda Luky Maya Mgbadolite Mirinda Nakuu Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui,29


### Single Divergent Individual Clustering Windows <a class = 'anchor' id = 'singledivergentindividualclusteringwindows'></a>

Let's look for different topologies. Starting with windows where a single individual is divergent to all others.

In [79]:
SDI_windows = complete_linkage_trees[(complete_linkage_trees['n_clusters'] == 2) & (complete_linkage_trees['n_single_clusters'] == 1)]

In [80]:
len(SDI_windows)

800

In [81]:
SDI_windows.head(5)

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition
11,chr10_8388608,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.144765,0.144765,0.093802,2,1,SeppToni/Akwaya-Jean Alfred Alice Andromeda At...
15,chr10_10485760,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.056945,0.056945,0.01485,2,1,Desmond/Akwaya-Jean Alfred Alice Andromeda Ath...
16,chr10_11010048,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.019138,0.019138,0.011459,2,1,Desmond/Akwaya-Jean Alfred Alice Andromeda Ath...
29,chr10_20971520,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.024704,0.024704,0.01044,2,1,Kidongo/Akwaya-Jean Alfred Alice Andromeda Ath...
30,chr10_21495808,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.0,0.017877,0.017877,0.005218,2,1,Kidongo/Akwaya-Jean Alfred Alice Andromeda Ath...


What is the distribution of individuals in the single individual clusters?

In [82]:
SDI_individuals = SDI_windows['window'].copy().to_frame('window')
SDI_individuals['SDI_ind'] = SDI_windows['cluster_composition'].str.split('/').str[0]
SDI_individuals.head(5)

,window,SDI_ind
11,chr10_8388608,SeppToni
15,chr10_10485760,Desmond
16,chr10_11010048,Desmond
29,chr10_20971520,Kidongo
30,chr10_21495808,Kidongo


In [83]:
SDI_individuals_summary = SDI_individuals.groupby(['SDI_ind']).size().to_frame('N')
SDI_individuals_summary.head(5)

,N
SDI_ind,
Akwaya-Jean,16
Alfred,21
Alice,9
Andromeda,12
Athanga,13


In [84]:
len(SDI_individuals_summary)

56

In [85]:
SDI_individuals_summary[SDI_individuals_summary['N'] == SDI_individuals_summary['N'].min()]

,N
SDI_ind,
Mgbadolite,1


In [86]:
SDI_individuals_summary[SDI_individuals_summary['N'] == SDI_individuals_summary['N'].max()]

,N
SDI_ind,
Luky,34


In [87]:
SDI_individuals_summary.mean()

N    14.285714
dtype: float64

There is some inter-individual variation in the number of windows where an individual is highly divergent. These windows may also vary considerably in their maximum depth so let's assess the range of max divergence per individual. We will also confirm that the individual is part of the pair with the maximum.

In [88]:
maxes_with_pair_IDs = comparisons.loc[comparisons.groupby('window')['divergence'].idxmax(), ['window', 'ind1', 'ind2', 'divergence']]
SDI_maxes_with_pair_IDs = maxes_with_pair_IDs[maxes_with_pair_IDs['window'].isin(SDI_windows['window'])]
SDI_maxes_with_pair_IDs = SDI_maxes_with_pair_IDs.merge(SDI_individuals, on = 'window')
SDI_maxes_with_pair_IDs.head(5)

,window,ind1,ind2,divergence,SDI_ind
0,chr10_100663296,Alfred,Lara,0.101497,Alfred
1,chr10_102236160,Frederike,Lara,0.031867,Lara
2,chr10_102760448,Bono,Bwamble,0.082444,Bwamble
3,chr10_10485760,Desmond,Julie-A959,0.056945,Desmond
4,chr10_106954752,Kumbuka,Washu,0.340326,Kumbuka


Let's check if there are any rows where the SDI individual does not match the most divergent comparison.

In [89]:
SDI_maxes_with_pair_IDs = SDI_maxes_with_pair_IDs[(SDI_maxes_with_pair_IDs['SDI_ind'] == SDI_maxes_with_pair_IDs['ind1']) | (SDI_maxes_with_pair_IDs['SDI_ind'] == SDI_maxes_with_pair_IDs['ind2'])]

In [90]:
len(SDI_maxes_with_pair_IDs)

800

Everything looks good! Let's identify the min, average, and max divergence score per individual in windows where they are highly divergent.

In [91]:
SDI_maxes_with_pair_IDs.head(5)

,window,ind1,ind2,divergence,SDI_ind
0,chr10_100663296,Alfred,Lara,0.101497,Alfred
1,chr10_102236160,Frederike,Lara,0.031867,Lara
2,chr10_102760448,Bono,Bwamble,0.082444,Bwamble
3,chr10_10485760,Desmond,Julie-A959,0.056945,Desmond
4,chr10_106954752,Kumbuka,Washu,0.340326,Kumbuka


In [92]:
SDI_n_per_ind = SDI_maxes_with_pair_IDs.groupby(['SDI_ind'])['divergence'].count()
SDI_min_per_ind = SDI_maxes_with_pair_IDs.groupby(['SDI_ind'])['divergence'].min()
SDI_mean_per_ind = SDI_maxes_with_pair_IDs.groupby(['SDI_ind'])['divergence'].mean()
SDI_max_per_ind = SDI_maxes_with_pair_IDs.groupby(['SDI_ind'])['divergence'].max()

In [93]:
SDI_stats_per_ind = pd.DataFrame({'N':SDI_n_per_ind, 'min':SDI_min_per_ind, 'mean':SDI_mean_per_ind, 'max':SDI_max_per_ind}).reset_index()
SDI_stats_per_ind.head(5)

,SDI_ind,N,min,mean,max
0,Akwaya-Jean,16,0.011237,0.049478,0.204380
1,Alfred,21,0.004539,0.055706,0.201247
2,Alice,9,0.008469,0.118712,0.502525
3,Andromeda,12,0.013257,0.050905,0.107627
4,Athanga,13,0.006031,0.126002,0.463006


Map on lineage for plotting.

In [94]:
lineage_dict = {'Bono':'ppn','Desmond':'ppn','Dzeeta':'ppn','Hermien':'ppn','Hortense':'ppn','Kombote':'ppn','Kosana':'ppn','Kumbuka':'ppn','Natalie':'ppn',
                'Akwaya-Jean':'pte', 'Damian':'pte', 'Julie-LWC21':'pte', 'Koto':'pte', 'Taweh':'pte',
                'Andromeda':'pts','Athanga':'pts','Bihati':'pts','Bwamble':'pts','Cindy-schwein':'pts','Cleo':'pts','Coco-chimp':'pts','Frederike':'pts','Ikuru':'pts','Kidongo':'pts','Maya':'pts','Mgbadolite':'pts','Nakuu':'pts','Tongo':'pts','Trixie':'pts','Vincent':'pts','Washu':'pts',
                'Alfred':'ptt','Blanquita':'ptt','Brigitta':'ptt','Cindy-troglodytes':'ptt','Doris':'ptt','Gamin':'ptt','Julie-A959':'ptt','Lara':'ptt','Luky':'ptt','Marlin':'ptt','Mirinda':'ptt','Negrita':'ptt','Tibe':'ptt','Ula':'ptt','Vaillant':'ptt','Yogui':'ptt',
                'Alice':'ptv','Berta':'ptv','Bosco':'ptv','Cindy-verus':'ptv','Clint':'ptv','Jimmie':'ptv','Koby':'ptv','Linda':'ptv','SeppToni':'ptv'}

In [95]:
SDI_stats_per_ind['lineage'] = SDI_stats_per_ind['SDI_ind'].map(lineage_dict)
SDI_stats_per_ind = SDI_stats_per_ind[['SDI_ind','lineage','N','min','mean','max']]
SDI_stats_per_ind.head(5)

,SDI_ind,lineage,N,min,mean,max
0,Akwaya-Jean,pte,16,0.011237,0.049478,0.204380
1,Alfred,ptt,21,0.004539,0.055706,0.201247
2,Alice,ptv,9,0.008469,0.118712,0.502525
3,Andromeda,pts,12,0.013257,0.050905,0.107627
4,Athanga,pts,13,0.006031,0.126002,0.463006


In [96]:
SDI_stats_per_ind['lineage'].unique()

array(['pte', 'ptt', 'ptv', 'pts', 'ppn'], dtype=object)

In [97]:
len(SDI_stats_per_ind[SDI_stats_per_ind['max'] > 0.05])

53

In [98]:
len(SDI_stats_per_ind[SDI_stats_per_ind['max'] > 0.25])

28

In [99]:
28/56

0.5

Save the dataframe for plotting and inclusion in supplemental data.

In [100]:
SDI_stats_per_ind.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/SDI_stats_per_ind.txt', sep = '\t', header = True, index = False)

### Lineage-Specific Clustering Windows <a class = 'anchor' id = 'lineagespecificclusteringwindows'></a>

Now let's look for windows where all members of a given lineage cluster separately from all other individuals. We'll write a function to do this and then pass in lists of IDs for the autosomes and chrX. We can use "startswith" here because the clusters are ordered from smallest to largest, thus, the smaller, lineage-specific cluster will be listed first.

In [101]:
def count_lineage_specific_clustering_windows(IDs, female_IDs):
    autosomal_windows = complete_linkage_trees[(complete_linkage_trees['n_clusters'] == 2) & (~complete_linkage_trees['window'].str.startswith('chrX')) & (complete_linkage_trees['cluster_composition'].str.startswith(IDs))]
    chrX_windows = complete_linkage_trees[(complete_linkage_trees['n_clusters'] == 2) & (complete_linkage_trees['window'].str.startswith('chrX')) & (complete_linkage_trees['cluster_composition'].str.startswith(female_IDs))]
    return len(autosomal_windows + chrX_windows)

In [102]:
ppn_IDs = 'Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/'
ppn_female_IDs = 'Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/'

pts_IDs = 'Andromeda Athanga Bihati Cindy-schwein Cleo Coco-chimp Frederike Ikuru Kidongo Maya Mgbadolite Nakuu Tongo Trixie Vincent Washu/'
pts_female_IDs = 'Andromeda Bihati Cindy-schwein Cleo Coco-chimp Frederike Ikuru Kidongo Maya Nakuu Trixie/'

ptt_IDs = 'Alfred Blanquita Brigitta Cindy-troglodytes Doris Gamin Julie-A959 Lara Luky Marlin Mirinda Negrita Tibe Ula Vaillant Yogui/'
ptt_female_IDs = 'Blanquita Cindy-troglodytes Doris Julie-A959 Lara Luky Marlin Mirinda Negrita Tibe Ula/'

ptv_IDs = 'Alice Berta Bosco Cindy-verus Clint Jimmie Koby Linda SeppToni/'
ptv_female_IDs = 'Alice Berta Cindy-verus Jimmie Linda/'

In [103]:
count_lineage_specific_clustering_windows(ppn_IDs, ppn_female_IDs)

339

In [104]:
count_lineage_specific_clustering_windows(pts_IDs, pts_female_IDs)

0

In [105]:
count_lineage_specific_clustering_windows(ptt_IDs, ptt_female_IDs)

0

In [106]:
count_lineage_specific_clustering_windows(ptv_IDs, ptv_female_IDs)

8

Let's do this manually for Nigeria-Cameroon chimpanzees. There is only one female; therefore, any lineage-specific chr X patterns are designated as rare contact pattern windows.

In [107]:
len(complete_linkage_trees[(complete_linkage_trees['n_clusters'] == 2) & (~complete_linkage_trees['window'].str.startswith('chrX')) & (complete_linkage_trees['cluster_composition'].str.startswith('Akwaya-Jean Damian Julie-LWC21 Koto Taweh/'))])

0

Grab the bonobo-chimpanzee clustering windows.

In [108]:
ppn_ptr_windows = complete_linkage_trees[(complete_linkage_trees['n_clusters'] == 2) & (~complete_linkage_trees['window'].str.startswith('chrX')) & (complete_linkage_trees['cluster_composition'].str.startswith('Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/')) | (complete_linkage_trees['n_clusters'] == 2) & (complete_linkage_trees['window'].str.startswith('chrX')) & (complete_linkage_trees['cluster_composition'].str.startswith('Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/'))]

In [109]:
len(ppn_ptr_windows)

339

Now let's take a closer look at the western chimpanzee windows.

In [110]:
western_chimpanzee_divergent_windows = complete_linkage_trees[(complete_linkage_trees['cluster_composition'] == 'Alice Berta Bosco Cindy-verus Clint Jimmie Koby Linda SeppToni/Akwaya-Jean Alfred Andromeda Athanga Bihati Blanquita Bono Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cleo Coco-chimp Damian Desmond Doris Dzeeta Frederike Gamin Hermien Hortense Ikuru Julie-A959 Julie-LWC21 Kidongo Kombote Kosana Koto Kumbuka Lara Luky Marlin Maya Mgbadolite Mirinda Nakuu Natalie Negrita Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui') | (complete_linkage_trees['cluster_composition'] == 'Alice Berta Cindy-verus Jimmie Linda/Andromeda Bihati Blanquita Cindy-schwein Cindy-troglodytes Cleo Coco-chimp Doris Dzeeta Frederike Hermien Hortense Ikuru Julie-A959 Julie-LWC21 Kidongo Kombote Kosana Kumbuka Lara Luky Marlin Maya Mirinda Nakuu Natalie Negrita Taweh Tibe Trixie Ula')]
western_chimpanzee_divergent_windows

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition
998,chr15_28835840,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.000993,0.007565,0.007565,0.004479,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
1020,chr15_40370176,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002281,0.009507,0.009507,0.006044,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
1798,chr1_210239488,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.00071,0.019751,0.019751,0.013096,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
2708,chr3_169345024,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002744,0.014599,0.014599,0.007954,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
3153,chr5_44564480,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.000428,0.011706,0.011706,0.004777,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
3154,chr5_45088768,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.000929,0.023077,0.023077,0.014528,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
3425,chr6_52428800,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.001113,0.009093,0.009093,0.003563,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...
3426,chr6_52953088,C2,C2,C1,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.006607,0.030093,0.030093,0.016993,2,0,Alice Berta Bosco Cindy-verus Clint Jimmie Kob...


Do any of these windows intersect interesting genes? First, create a BED file of the windows above and then create a pybedtools object.

In [111]:
western_chimpanzee_divergent_windows_BED = pd.DataFrame()
western_chimpanzee_divergent_windows_BED['window_split'] = western_chimpanzee_divergent_windows['window']
western_chimpanzee_divergent_windows_BED = western_chimpanzee_divergent_windows_BED['window_split'].str.split('_', expand=True)
western_chimpanzee_divergent_windows_BED.rename(columns = {0:'chr', 1:'window_start'}, inplace = True)
western_chimpanzee_divergent_windows_BED['window_start'] = western_chimpanzee_divergent_windows_BED['window_start'].astype(int)
western_chimpanzee_divergent_windows_BED['window_end'] = western_chimpanzee_divergent_windows_BED['window_start'] + 1048576
western_chimpanzee_divergent_windows_BED = western_chimpanzee_divergent_windows_BED.sort_values(by = ['chr','window_start'])
western_chimpanzee_divergent_windows_BED.head(10)

,chr,window_start,window_end
1798,chr1,210239488,211288064
998,chr15,28835840,29884416
1020,chr15,40370176,41418752
2708,chr3,169345024,170393600
3153,chr5,44564480,45613056
3154,chr5,45088768,46137344
3425,chr6,52428800,53477376
3426,chr6,52953088,54001664


In [112]:
western_chimpanzee_divergent_windows_pbtBED = pybedtools.BedTool().from_dataframe(western_chimpanzee_divergent_windows_BED).sort()
western_chimpanzee_divergent_windows_pbtBED.head()

chr1	210239488	211288064
 chr15	28835840	29884416
 chr15	40370176	41418752
 chr3	169345024	170393600
 chr5	44564480	45613056
 chr5	45088768	46137344
 chr6	52428800	53477376
 chr6	52953088	54001664
 

Now read in the panTro6 genes as a pybedtools object.

In [113]:
genes_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/annotations/panTro6_genes.bed')
genes_pbtBED.head(5)

chr1	344	12299	NM_001280424.1	INTS11
 chr1	12400	16513	XM_016952033.2	CPTP
 chr1	17887	23426	XM_003307748.4	TAS1R3
 chr1	22943	36942	XM_016958290.2	DVL1
 chr1	39619	45507	NM_001280245.1	MXRA8
 

Intersect the western chimpanzee divergent windows with panTro6 genes.

In [114]:
western_chimpanzee_divergent_windows_genes_intersect = western_chimpanzee_divergent_windows_pbtBED.intersect(genes_pbtBED, wa = True, wb = True).to_dataframe(names=['window_chr','window_start','window_end','gene_chr','gene_start','gene_end','gene_transcript','gene'])
western_chimpanzee_divergent_windows_genes_intersect.head(5)

,window_chr,window_start,window_end,gene_chr,gene_start,gene_end,gene_transcript,gene
0,chr1,210239488,211288064,chr1,210270967,210286436,XM_016941282.1,GGPS1
1,chr1,210239488,211288064,chr1,210393829,210457855,XM_016941305.2,B3GALNT2
2,chr1,210239488,211288064,chr1,210504199,210604057,XM_009441703.3,GNG4
3,chr1,210239488,211288064,chr1,210927111,211015721,XM_001156001.5,NID1
4,chr1,210239488,211288064,chr1,211167578,211234322,XM_016927789.2,ERO1B


In [115]:
len(western_chimpanzee_divergent_windows_genes_intersect['gene'].drop_duplicates())

52

Let's grab the identities of the pair for the most divergent comparison per western chimpanzee divergent window.

In [116]:
comparisons[(comparisons['window'] == 'chr1_210239488')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                     Gamin
ind2                  SeppToni
lineages               ptt-ptv
chr                       chr1
window_start         210239488
window          chr1_210239488
mse                   0.013139
spearman              0.980249
divergence            0.019751
seq_diff                  2283
Name: 4472871, dtype: object

In [117]:
comparisons[(comparisons['window'] == 'chr3_169345024')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                    Jimmie
ind2                     Tongo
lineages               pts-ptv
chr                       chr3
window_start         169345024
window          chr3_169345024
mse                   0.003829
spearman              0.985401
divergence            0.014599
seq_diff                  2550
Name: 5002410, dtype: object

In [118]:
comparisons[(comparisons['window'] == 'chr5_44564480')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                  Hermien
ind2                     Koby
lineages              ppn-ptr
chr                      chr5
window_start         44564480
window          chr5_44564480
mse                  0.009547
spearman             0.988294
divergence           0.011706
seq_diff                 2870
Name: 4543436, dtype: object

In [119]:
comparisons[(comparisons['window'] == 'chr5_45088768')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1            Cindy-schwein
ind2                   Jimmie
lineages              pts-ptv
chr                      chr5
window_start         45088768
window          chr5_45088768
mse                  0.006429
spearman             0.976923
divergence           0.023077
seq_diff                 2542
Name: 2625131, dtype: object

In [120]:
comparisons[(comparisons['window'] == 'chr6_52428800')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                    Bosco
ind2               Julie-A959
lineages              ptt-ptv
chr                      chr6
window_start         52428800
window          chr6_52428800
mse                  0.005152
spearman             0.990907
divergence           0.009093
seq_diff                 2202
Name: 2064653, dtype: object

In [121]:
comparisons[(comparisons['window'] == 'chr6_52953088')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                    Bosco
ind2                  Kumbuka
lineages              ppn-ptr
chr                      chr6
window_start         52953088
window          chr6_52953088
mse                  0.009424
spearman             0.969907
divergence           0.030093
seq_diff                 3791
Name: 2094537, dtype: object

In [122]:
comparisons[(comparisons['window'] == 'chr15_28835840')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                     Bosco
ind2                   Mirinda
lineages               ptt-ptv
chr                      chr15
window_start          28835840
window          chr15_28835840
mse                   0.001035
spearman              0.992435
divergence            0.007565
seq_diff                  1963
Name: 2121992, dtype: object

In [123]:
comparisons[(comparisons['window'] == 'chr15_40370176')].sort_values(by = 'divergence', ascending = False).iloc[0]

ind1                 Andromeda
ind2                     Clint
lineages               pts-ptv
chr                      chr15
window_start          40370176
window          chr15_40370176
mse                   0.002546
spearman              0.990493
divergence            0.009507
seq_diff                  2485
Name: 750168, dtype: object

### Multiple Divergent Individuals Clustering Windows <a class = 'anchor' id = 'multipledivergentindividualsclusteringwindows'></a>

Thus far, we have identified 800 single divergent individual clustering windows, 339 bonobo-chimpanzee clustering windows, and 8 Western chimpanzee clustering windows. However, this still leaves 2475 two cluster windows: 3622 - (800 + 339 + 8). Based on the autosomal and X chromosome dataframes above, the remaining two cluster windows appear to be topologies where a smaller cluster of individuals from the same lineage or multiple lineages cluster together.

In [124]:
MDI_windows = complete_linkage_trees[complete_linkage_trees['n_clusters'] == 2]
windows_to_exclude = pd.concat([SDI_windows, ppn_ptr_windows, western_chimpanzee_divergent_windows])['window']
MDI_windows = MDI_windows[~MDI_windows['window'].isin(windows_to_exclude)]

In [125]:
len(MDI_windows)

2475

What is the range of size and lineage composition of the smaller clusters in the MDIs?

In [126]:
MDI_windows_small_cluster = MDI_windows['cluster_composition'].str.split('/').apply(lambda x: x[0].split())
MDI_windows_small_cluster

1                [Cindy-troglodytes, Doris, Luky, Marlin]
2                [Cindy-troglodytes, Doris, Luky, Marlin]
3                              [Alice, Cindy-verus, Luky]
6       [Alfred, Andromeda, Bono, Coco-chimp, Doris, D...
8       [Bono, Desmond, Dzeeta, Hermien, Hortense, Kos...
                              ...                        
4411                                    [Dzeeta, Kumbuka]
4414                              [Bihati, Cindy-schwein]
4415    [Cindy-troglodytes, Luky, Marlin, Negrita, Tib...
4418    [Blanquita, Dzeeta, Hermien, Hortense, Kombote...
4419    [Andromeda, Cindy-schwein, Cindy-troglodytes, ...
Name: cluster_composition, Length: 2475, dtype: object

In [127]:
MDI_windows_small_cluster_counts = MDI_windows_small_cluster.apply(lambda x: len(x)).to_frame('small_cluster_N')
MDI_windows_small_cluster_counts_summary = MDI_windows_small_cluster_counts['small_cluster_N'].value_counts().to_frame('N').reset_index().rename(columns = {'index':'small_cluster_N'}).sort_values(by=['small_cluster_N'])
MDI_windows_small_cluster_counts_summary

,small_cluster_N,N
0,2,436
1,3,264
2,4,166
3,5,154
4,6,122
8,7,98
7,8,101
11,9,68
5,10,116
6,11,102


In [128]:
MDI_windows_small_cluster_counts.median()

small_cluster_N    7.0
dtype: float64

In [129]:
MDI_windows_small_cluster_counts_summary.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/MDI_windows_small_cluster_counts_summary.txt', sep = '\t', header = True, index = False)

In [130]:
MDI_windows_small_cluster_composition = MDI_windows_small_cluster.map(lambda x: [lineage_dict.get(item, item) for item in x]).to_frame('composition')
MDI_windows_small_cluster_composition.head(5)

,composition
1,"[ptt, ptt, ptt, ptt]"
2,"[ptt, ptt, ptt, ptt]"
3,"[ptv, ptv, ptt]"
6,"[ptt, pts, ppn, pts, ptt, ppn, ptt, ppn, ppn, ..."
8,"[ppn, ppn, ppn, ppn, ppn, ppn, ppn, ppn]"


In [131]:
pan_lineages = ['ppn','pte','pts','ptt','ptv']
for lineage in pan_lineages:
    MDI_windows_small_cluster_composition[lineage + '_present'] = MDI_windows_small_cluster_composition['composition'].apply(lambda x: lineage in x)
MDI_windows_small_cluster_composition.head(5)

,composition,ppn_present,pte_present,pts_present,ptt_present,ptv_present
1,"[ptt, ptt, ptt, ptt]",False,False,False,True,False
2,"[ptt, ptt, ptt, ptt]",False,False,False,True,False
3,"[ptv, ptv, ptt]",False,False,False,True,True
6,"[ptt, pts, ppn, pts, ptt, ppn, ptt, ppn, ppn, ...",True,True,True,True,False
8,"[ppn, ppn, ppn, ppn, ppn, ppn, ppn, ppn]",True,False,False,False,False


In [132]:
MDI_windows_small_cluster_composition[['ppn_present','pte_present','pts_present','ptt_present','ptv_present']].to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/MDI_windows_small_cluster_composition.txt', sep = '\t', header = True, index = False)

### Chr21 Window Topologies <a class = 'anchor' id = 'chr21windowtopologies'></a>

Now let's look at the topologies along an entire chromosome for Figure 2.

In [133]:
autosomal_complete_linkage_trees[autosomal_complete_linkage_trees['window'].str.startswith('chr21_')]

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition
1915,chr21_1048576,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.007273,0.101743,0.101743,0.018713,2,0,Cindy-troglodytes Kosana/Akwaya-Jean Alfred Al...
1916,chr21_1572864,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.009908,0.092433,0.092433,0.030027,2,0,Cindy-troglodytes Kosana/Akwaya-Jean Alfred Al...
1917,chr21_2097152,C3,C1,C1,C2,C3,C1,C3,C3,C3,C1,C1,C1,C3,C3,C3,C2,C1,C2,C3,C3,C3,C3,C2,C3,C3,C3,C2,C1,C2,C2,C2,C1,C3,C3,C3,C3,C2,C1,C3,C3,C1,C2,C3,C3,C3,C1,C1,C3,C1,C1,C2,C1,C3,C2,C3,C1,0.008533,0.016795,0.016795,0.013133,3,0,Andromeda Cleo Coco-chimp Frederike Ikuru Juli...
1918,chr21_2621440,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.001896,0.012374,0.012374,0.008402,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1919,chr21_3145728,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.000000,0.058120,0.058120,0.038627,2,1,Hortense/Akwaya-Jean Alfred Alice Andromeda At...
1920,chr21_3670016,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005336,0.025069,0.025069,0.007850,2,0,Bono Desmond Dzeeta Hortense Kombote Kosana Ku...
1921,chr21_4194304,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,0.001153,0.017037,0.017037,0.009157,2,0,Blanquita Cindy-troglodytes Ula/Akwaya-Jean Al...
1922,chr21_4718592,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.000000,0.056423,0.056423,0.021865,2,1,SeppToni/Akwaya-Jean Alfred Alice Andromeda At...
1923,chr21_5242880,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.000000,0.119308,0.119308,0.041856,2,1,SeppToni/Akwaya-Jean Alfred Alice Andromeda At...
1924,chr21_5767168,C1,C1,C1,C1,C1,C1,C2,C1,C1,C1,C1,C2,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C2,C1,C1,C1,C1,C1,C2,C1,C1,C1,C1,C1,C1,C1,C1,C1,C0,C1,C1,C2,C1,C1,C1,C1,C1,C1,C1,C2,C1,C1,C1,C2,C2,C1,0.049416,0.123899,0.123899,0.108459,3,1,Luky/Bihati Bwamble Frederike Julie-A959 Mgbad...


Let's subset the species divergent windows for this chromosome.

In [134]:
autosomal_complete_linkage_trees[(autosomal_complete_linkage_trees['window'].str.startswith('chr21_')) & (autosomal_complete_linkage_trees['cluster_composition'] == 'Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Doris Frederike Gamin Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Koto Lara Linda Luky Marlin Maya Mgbadolite Mirinda Nakuu Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui')]

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition
1918,chr21_2621440,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.001896,0.012374,0.012374,0.008402,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1935,chr21_11534336,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.000866,0.006946,0.006946,0.004649,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1948,chr21_19398656,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.003290,0.006391,0.006391,0.004359,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1949,chr21_19922944,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002029,0.024514,0.024514,0.005010,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1950,chr21_20447232,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002951,0.028111,0.028111,0.016831,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1957,chr21_24117248,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005625,0.034644,0.034644,0.011145,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...
1964,chr21_27787264,C1,C1,C1,C1,C1,C1,C1,C1,C2,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C2,C1,C2,C1,C1,C2,C2,C1,C1,C1,C1,C1,C1,C2,C2,C1,C2,C1,C1,C1,C1,C1,C1,C1,C1,C2,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,C1,0.021100,0.048625,0.048625,0.031750,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...


### Window Topology Divergence Differences <a class = 'anchor' id = 'windowtopologydivergencedifferences'></a>

Does the distribution for window maxima vary by the different topologies?

In [135]:
SDI_window_maxes = maxes[maxes['window'].isin(SDI_windows['window'])]['max']
MDI_window_maxes = maxes[maxes['window'].isin(MDI_windows['window'])]['max']
ppn_ptr_window_maxes = maxes[maxes['window'].isin(ppn_ptr_windows['window'])]['max']

In [136]:
len(SDI_window_maxes)

800

In [137]:
len(MDI_window_maxes )

2475

In [138]:
len(ppn_ptr_window_maxes)

339

In [139]:
SDI_window_maxes.mean()

0.06748740115086496

In [140]:
MDI_window_maxes.mean()

0.053014378694518735

In [141]:
ppn_ptr_window_maxes.mean()

0.049107185857274284

In [142]:
kruskal(SDI_window_maxes, MDI_window_maxes, ppn_ptr_window_maxes)

KruskalResult(statistic=31.09985751100612, pvalue=1.7650286021802972e-07)

Save these results for visualization.

In [143]:
SDI_window_maxes.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/SDI_window_maxes.txt', sep = '\t', header = False, index = False)
MDI_window_maxes.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/MDI_window_maxes.txt', sep = '\t', header = False, index = False)
ppn_ptr_window_maxes.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/ppn_ptr_window_maxes.txt', sep = '\t', header = False, index = False)

### Three, Four, and Five Cluster Window Topologies <a class = 'anchor' id = 'threefourfiveclusterwindowtopologies'></a>

Let's take a peak at the three, four, and five cluster windows. Change the maximum column width display setting to get a better look.

In [144]:
pd.options.display.max_colwidth = 500

In [145]:
complete_linkage_trees[complete_linkage_trees['n_clusters'] == 3][['window','cluster_composition']].head(10)

,window,cluster_composition
0,chr10_1572864,Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Andromeda Athanga Blanquita Bwamble Cindy-schwein Cleo Coco-chimp Doris Kidongo Lara Maya Mgbadolite Mirinda Nakuu Negrita Tongo Trixie Vincent Washu Yogui/Akwaya-Jean Alfred Alice Berta Bihati Bosco Brigitta Cindy-troglodytes Cindy-verus Clint Damian Frederike Gamin Ikuru Jimmie Julie-A959 Julie-LWC21 Koby Koto Linda Luky Marlin SeppToni Taweh Tibe Ula Vaillant
7,chr10_5242880,Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Andromeda Brigitta Coco-chimp Frederike Julie-LWC21 Lara Maya Mirinda Nakuu Tongo Vincent/Akwaya-Jean Alfred Alice Athanga Berta Bihati Blanquita Bosco Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Damian Doris Gamin Ikuru Jimmie Julie-A959 Kidongo Koby Koto Linda Luky Marlin Mgbadolite Negrita SeppToni Taweh Tibe Trixie Ula Vaillant Washu Yogui
13,chr10_9437184,Ikuru/Cleo Kidongo Maya Mgbadolite Washu/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bono Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Clint Coco-chimp Damian Desmond Doris Dzeeta Frederike Gamin Hermien Hortense Jimmie Julie-A959 Julie-LWC21 Koby Kombote Kosana Koto Kumbuka Lara Linda Luky Marlin Mirinda Nakuu Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Yogui
20,chr10_13107200,Frederike Julie-A959 Tibe Ula/Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie Negrita/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bosco Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Doris Gamin Ikuru Jimmie Julie-LWC21 Kidongo Koby Koto Lara Linda Luky Marlin Maya Mgbadolite Mirinda Nakuu SeppToni Taweh Tongo Trixie Vaillant Vincent Washu Yogui
21,chr10_15728640,Kosana Kumbuka/Akwaya-Jean Andromeda Brigitta Cindy-troglodytes Damian Gamin Ikuru Julie-A959 Julie-LWC21 Koto Lara Nakuu Ula Yogui/Alfred Alice Athanga Berta Bihati Blanquita Bono Bosco Bwamble Cindy-schwein Cindy-verus Cleo Clint Coco-chimp Desmond Doris Dzeeta Frederike Hermien Hortense Jimmie Kidongo Koby Kombote Linda Luky Marlin Maya Mgbadolite Mirinda Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Vaillant Vincent Washu
22,chr10_16252928,Julie-A959/Bono Brigitta Desmond Dzeeta Gamin Hermien Hortense Kombote Kosana Kumbuka Linda Mirinda Natalie/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bosco Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Doris Frederike Ikuru Jimmie Julie-LWC21 Kidongo Koby Koto Lara Luky Marlin Maya Mgbadolite Nakuu Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui
31,chr10_22020096,Bono Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Alfred Andromeda Athanga Blanquita Brigitta Coco-chimp Damian Ikuru Julie-LWC21 Kidongo Koto Lara Luky Mgbadolite Taweh Tongo Trixie Ula/Akwaya-Jean Alice Berta Bihati Bosco Bwamble Cindy-schwein Cindy-troglodytes Cindy-verus Cleo Clint Desmond Doris Frederike Gamin Jimmie Julie-A959 Koby Linda Marlin Maya Mirinda Nakuu Negrita SeppToni Tibe Vaillant Vincent Washu Yogui
32,chr10_22544384,Lara/Cindy-schwein Gamin Hortense Julie-A959 Kosana Kumbuka Vaillant/Akwaya-Jean Alfred Alice Andromeda Athanga Berta Bihati Blanquita Bono Bosco Brigitta Bwamble Cindy-troglodytes Cindy-verus Cleo Clint Coco-chimp Damian Desmond Doris Dzeeta Frederike Hermien Ikuru Jimmie Julie-LWC21 Kidongo Koby Kombote Koto Linda Luky Marlin Maya Mgbadolite Mirinda Nakuu Natalie Negrita SeppToni Taweh Tibe Tongo Trixie Ula Vincent Washu Yogui
33,chr10_23068672,Lara/Alfred Athanga Brigitta Cindy-schwein Cleo Coco-chimp Ikuru Marlin Mgbadolite Mirinda Negrita Trixie Vaillant Vincent/Akwaya-Jean Alice Andromeda Berta Bihati Blanquita Bono Bosco Bwamble Cindy-troglodytes Cindy-verus Clint Damian Desmond Doris Dzeeta Frederike Gamin Hermien Hortense Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Kombote Kosana Kot

In [146]:
complete_linkage_trees[complete_linkage_trees['n_clusters'] == 4][['window','cluster_composition']].head(30)

,window,cluster_composition
49,chr10_31457280,Blanquita Bosco Coco-chimp Lara Trixie/Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Akwaya-Jean Alice Berta Cindy-verus Clint Damian Jimmie Julie-LWC21 Koby Koto Linda SeppToni Taweh Tibe Tongo/Alfred Andromeda Athanga Bihati Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cleo Doris Frederike Gamin Ikuru Julie-A959 Kidongo Luky Marlin Maya Mgbadolite Mirinda Nakuu Negrita Ula Vaillant Vincent Washu Yogui
101,chr10_65536000,Doris Ula Yogui/Athanga Bono Cindy-schwein Cleo Coco-chimp Desmond Dzeeta Hermien Hortense Kidongo Kombote Kosana Kumbuka Nakuu Natalie Washu/Akwaya-Jean Alice Bosco Gamin Ikuru Jimmie Julie-A959 Julie-LWC21 Koby Koto Lara Linda Marlin SeppToni Tibe Vaillant Vincent/Alfred Andromeda Berta Bihati Blanquita Brigitta Bwamble Cindy-troglodytes Cindy-verus Clint Damian Frederike Luky Maya Mgbadolite Mirinda Negrita Taweh Tongo Trixie
199,chr10_121110528,Brigitta Hermien/Doris Gamin SeppToni/Athanga Bihati Cindy-schwein Coco-chimp Damian Ikuru Julie-A959 Nakuu Tibe Vaillant/Akwaya-Jean Alfred Alice Andromeda Berta Blanquita Bono Bosco Bwamble Cindy-troglodytes Cindy-verus Cleo Clint Desmond Dzeeta Frederike Hortense Jimmie Julie-LWC21 Kidongo Koby Kombote Kosana Koto Kumbuka Lara Linda Luky Marlin Maya Mgbadolite Mirinda Natalie Negrita Taweh Tongo Trixie Ula Vincent Washu Yogui
212,chr11_2097152,Cleo Ikuru/Coco-chimp Kidongo Nakuu/Alfred Blanquita Cindy-troglodytes Gamin Julie-A959 Lara Marlin Negrita Ula Yogui/Akwaya-Jean Alice Andromeda Athanga Berta Bihati Bono Bosco Brigitta Bwamble Cindy-schwein Cindy-verus Clint Damian Desmond Doris Dzeeta Frederike Hermien Hortense Jimmie Julie-LWC21 Koby Kombote Kosana Koto Kumbuka Linda Luky Maya Mgbadolite Mirinda Natalie SeppToni Taweh Tibe Tongo Trixie Vaillant Vincent Washu
282,chr11_41418752,Mirinda Negrita/Bono Dzeeta Hortense Kombote Kosana/Alice Berta Bosco Cindy-verus Koby Linda SeppToni/Akwaya-Jean Alfred Andromeda Athanga Bihati Blanquita Brigitta Bwamble Cindy-schwein Cindy-troglodytes Cleo Clint Coco-chimp Damian Desmond Doris Frederike Gamin Hermien Ikuru Jimmie Julie-A959 Julie-LWC21 Kidongo Koto Kumbuka Lara Luky Marlin Maya Mgbadolite Nakuu Natalie Taweh Tibe Tongo Trixie Ula Vaillant Vincent Washu Yogui
358,chr11_87031808,Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Brigitta Doris Julie-LWC21 Lara Luky Marlin Negrita Tibe Ula Vaillant/Andromeda Athanga Bihati Bwamble Cindy-schwein Cleo Coco-chimp Ikuru Julie-A959 Kidongo Mgbadolite Nakuu Tongo Trixie Vincent Yogui/Akwaya-Jean Alfred Alice Berta Blanquita Bosco Cindy-troglodytes Cindy-verus Clint Damian Frederike Gamin Jimmie Koby Koto Linda Maya Mirinda SeppToni Taweh Washu
584,chr12_95420416,Akwaya-Jean Alfred Koto Nakuu Taweh/Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Andromeda Athanga Cindy-schwein Cleo Coco-chimp Ikuru Lara Marlin Negrita Tibe Tongo Trixie/Alice Berta Bihati Blanquita Bosco Brigitta Bwamble Cindy-troglodytes Cindy-verus Clint Damian Doris Frederike Gamin Jimmie Julie-A959 Julie-LWC21 Kidongo Koby Linda Luky Maya Mgbadolite Mirinda SeppToni Ula Vaillant Vincent Washu Yogui
690,chr13_26214400,Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Alice Berta Bosco Brigitta Cindy-verus Cleo Clint Jimmie Julie-A959 Koby Linda Nakuu SeppToni/Alfred Athanga Bihati Blanquita Bwamble Cindy-schwein Coco-chimp Kidongo Maya Mgbadolite Mirinda Negrita Tongo Trixie Ula Vincent Yogui/Akwaya-Jean Andromeda Cindy-troglodytes Damian Doris Frederike Gamin Ikuru Julie-LWC21 Koto Lara Luky Marlin Taweh Tibe Vaillant Washu
757,chr13_65011712,Cindy-troglodytes/Akwaya-Jean Alfred Brigitta Damian Kidongo Koto Taweh Vaillant/Berta Bono Desmond Dzeeta Gamin Hermien Hortense Jimmie Julie-LWC21 Koby Kombote Kosana Kumbuka Luky Marlin Natalie Negrita Ula Yogui/Alice Andromeda Athanga Bihati Blanquita Bosco Bwamble Cindy-schwein Cindy-verus Cleo Clint Coco-chimp Doris Frederike Ikur

In [147]:
complete_linkage_trees[complete_linkage_trees['n_clusters'] == 5][['window','cluster_composition']].head(4)

,window,cluster_composition
497,chr12_42467328,Hermien Kombote Vaillant/Bono Brigitta Julie-LWC21 Koby Kosana Koto Linda Yogui/Alice Berta Bosco Cindy-verus Clint Desmond Jimmie Natalie SeppToni/Alfred Bihati Bwamble Cindy-schwein Doris Frederike Gamin Ikuru Kidongo Luky Mgbadolite Nakuu Tongo Trixie Vincent Washu/Akwaya-Jean Andromeda Athanga Blanquita Cindy-troglodytes Cleo Coco-chimp Damian Dzeeta Hortense Julie-A959 Kumbuka Lara Marlin Maya Mirinda Negrita Taweh Tibe Ula
1494,chr1_12058624,Athanga/Cindy-troglodytes Frederike Maya/Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Alice Bosco Cindy-schwein Cindy-verus Cleo Clint Coco-chimp Doris Ikuru Jimmie Lara Linda Marlin Mgbadolite Nakuu SeppToni Taweh Ula Vincent/Akwaya-Jean Alfred Andromeda Berta Bihati Blanquita Brigitta Bwamble Damian Gamin Julie-A959 Julie-LWC21 Kidongo Koby Koto Luky Mirinda Negrita Tibe Tongo Trixie Vaillant Washu Yogui
2817,chr4_39321600,Brigitta Cleo Negrita Vincent/Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka/Bihati Blanquita Kidongo Marlin Mirinda Trixie Ula Vaillant Yogui/Alfred Alice Berta Bosco Cindy-troglodytes Cindy-verus Clint Jimmie Koby Linda SeppToni Taweh/Akwaya-Jean Andromeda Athanga Bwamble Cindy-schwein Coco-chimp Damian Doris Frederike Gamin Ikuru Julie-A959 Julie-LWC21 Koto Lara Luky Maya Mgbadolite Nakuu Natalie Tibe Tongo Washu
2889,chr4_78118912,Brigitta/Bihati Cindy-schwein Cleo Coco-chimp Frederike Ikuru Tibe Tongo/Bono Desmond Dzeeta Hermien Hortense Kombote Kosana Kumbuka Natalie/Akwaya-Jean Alice Cindy-troglodytes Damian Julie-A959 Koto Luky Marlin Mirinda SeppToni Taweh/Alfred Andromeda Athanga Berta Blanquita Bosco Bwamble Cindy-verus Clint Doris Gamin Jimmie Julie-LWC21 Kidongo Koby Lara Linda Maya Mgbadolite Nakuu Negrita Trixie Ula Vaillant Vincent Washu Yogui


Reset the display setting.

In [148]:
pd.options.display.max_colwidth = 50

Let's map the topological classification of each window as well as genes in that window onto the hierarchical clustering dataframe. First, topology classification.

In [149]:
def topology(complete_linkage_trees):
    if complete_linkage_trees['window'] in SDI_windows['window'].values:
        return 'Highly Divergent Individual'
    elif complete_linkage_trees['window'] in MDI_windows['window'].values:
        return 'Multiple Divergent Individuals'
    elif complete_linkage_trees['window'] in ppn_ptr_windows['window'].values:
        return 'Bonobo-Chimpanzee Clustering'
    elif complete_linkage_trees['window'] in western_chimpanzee_divergent_windows['window'].values:
        return 'Western Chimpanzee Clustering'
    elif complete_linkage_trees['n_clusters'] == 3:
        return 'Three Cluster'
    elif complete_linkage_trees['n_clusters'] == 4:
        return 'Four Cluster'
    elif complete_linkage_trees['n_clusters'] == 5:
        return 'Five Cluster'
    else:
        return None

complete_linkage_trees['topology'] = complete_linkage_trees.apply(topology, axis = 1)

In [150]:
complete_linkage_trees.head(5)

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition,topology
0,chr10_1572864,C3,C3,C3,C2,C2,C3,C3,C2,C1,C3,C3,C2,C2,C3,C3,C2,C3,C2,C3,C1,C2,C1,C3,C3,C1,C1,C3,C3,C3,C3,C2,C3,C1,C1,C3,C1,C2,C3,C3,C3,C2,C2,C2,C2,C1,C2,C3,C3,C3,C2,C2,C3,C3,C2,C2,C2,0.014522,0.091748,0.091748,0.067514,3,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...,Three Cluster
1,chr10_2097152,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.038838,0.269334,0.269334,0.115208,2,0,Cindy-troglodytes Doris Luky Marlin/Akwaya-Jea...,Multiple Divergent Individuals
2,chr10_2621440,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005674,0.012824,0.012824,0.006737,2,0,Cindy-troglodytes Doris Luky Marlin/Akwaya-Jea...,Multiple Divergent Individuals
3,chr10_3145728,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002904,0.010145,0.010145,0.006418,2,0,Alice Cindy-verus Luky/Akwaya-Jean Alfred Andr...,Multiple Divergent Individuals
4,chr10_3670016,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.00514,0.096248,0.096248,0.04808,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...,Bonobo-Chimpanzee Clustering


On to genes per window. First, read in an intersection of the panTro6 windows used in analysis and genes BED files.

In [151]:
windows_genes_intersect = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/annotations/windows_genes_intersect.bed', sep = '\t', names = ['window_chr','window_start','window_end','gene_chr','gene_start','gene_end','gene_transcript','gene'])
windows_genes_intersect['window'] = windows_genes_intersect['window_chr'] + '_' + windows_genes_intersect['window_start'].astype(str)
windows_genes_intersect.head(5)

,window_chr,window_start,window_end,gene_chr,gene_start,gene_end,gene_transcript,gene,window
0,chr1,1048576,2097152,chr1,1088438,1096220,XM_513729.6,PEX10,chr1_1048576
1,chr1,1048576,2097152,chr1,1220059,1221565,XM_009446771.2,HES5,chr1_1048576
2,chr1,1048576,2097152,chr1,1248417,1256443,XM_009446795.3,TNFRSF14,chr1_1048576
3,chr1,1048576,2097152,chr1,1279618,1284329,XM_001151207.3,FAM213B,chr1_1048576
4,chr1,1048576,2097152,chr1,1606738,1608184,XM_524851.6,ACTRT2,chr1_1048576


Next, define a function to create an alphabetized list of genes per window.

In [152]:
def create_gene_list_per_window(window):
    sorted_genes = sorted(window)
    genes = [gene for gene in sorted_genes]
    return ', '.join(genes)

In [153]:
genes_per_window = windows_genes_intersect.groupby('window')['gene'].apply(create_gene_list_per_window).reset_index()
genes_per_window.head(5)

,window,gene
0,chr10_100139008,"CFAP58, GSTO2, ITPRIP, SORCS3"
1,chr10_100663296,SORCS3
2,chr10_101711872,SORCS1
3,chr10_102236160,SORCS1
4,chr10_102760448,SORCS1


In [154]:
complete_linkage_trees_with_genes = pd.merge(complete_linkage_trees, genes_per_window, on = ['window'])
complete_linkage_trees_with_genes = complete_linkage_trees_with_genes.rename(columns = {'gene':'genes'})
complete_linkage_trees_with_genes.head(5)

,window,Akwaya-Jean,Alfred,Alice,Andromeda,Athanga,Berta,Bihati,Blanquita,Bono,Bosco,Brigitta,Bwamble,Cindy-schwein,Cindy-troglodytes,Cindy-verus,Cleo,Clint,Coco-chimp,Damian,Desmond,Doris,Dzeeta,Frederike,Gamin,Hermien,Hortense,Ikuru,Jimmie,Julie-A959,Julie-LWC21,Kidongo,Koby,Kombote,Kosana,Koto,Kumbuka,Lara,Linda,Luky,Marlin,Maya,Mgbadolite,Mirinda,Nakuu,Natalie,Negrita,SeppToni,Taweh,Tibe,Tongo,Trixie,Ula,Vaillant,Vincent,Washu,Yogui,node_0,node_1,node_2,node_3,n_clusters,n_single_clusters,cluster_composition,topology,genes
0,chr10_1572864,C3,C3,C3,C2,C2,C3,C3,C2,C1,C3,C3,C2,C2,C3,C3,C2,C3,C2,C3,C1,C2,C1,C3,C3,C1,C1,C3,C3,C3,C3,C2,C3,C1,C1,C3,C1,C2,C3,C3,C3,C2,C2,C2,C2,C1,C2,C3,C3,C3,C2,C2,C3,C3,C2,C2,C2,0.014522,0.091748,0.091748,0.067514,3,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...,Three Cluster,"ADARB2, LOC112204541, LOC736356"
1,chr10_2621440,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.005674,0.012824,0.012824,0.006737,2,0,Cindy-troglodytes Doris Luky Marlin/Akwaya-Jea...,Multiple Divergent Individuals,"LOC107976892, PFKP, PITRM1"
2,chr10_3145728,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.002904,0.010145,0.010145,0.006418,2,0,Alice Cindy-verus Luky/Akwaya-Jean Alfred Andr...,Multiple Divergent Individuals,"KLF6, LOC107976892, PFKP, PITRM1"
3,chr10_3670016,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.00514,0.096248,0.096248,0.04808,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...,Bonobo-Chimpanzee Clustering,KLF6
4,chr10_4194304,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C1,C2,C2,C1,C1,C2,C2,C2,C2,C2,C2,C1,C1,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C1,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,C2,0.001365,0.012365,0.012365,0.007215,2,0,Bono Desmond Dzeeta Hermien Hortense Kombote K...,Bonobo-Chimpanzee Clustering,"AKR1C2, AKR1C3, AKR1E2, LOC100607996"


Save this dataframe.

In [155]:
complete_linkage_trees_with_genes.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/window_topologies/complete_linkage_trees_with_topologies_and_genes.txt', sep = '\t', header = True, index = False)

## Bonobo-Chimpanzee Windows <a class = 'anchor' id = 'bonobochimpanzeewindows'></a>

Let's take a closer look at the 339 windows with bonobo-chimpanzee clustering. Start with some basic statistics on the divergence scores in these windows.

In [156]:
def get_ppn_ptr_window_stats(window):
    stats = []
    subset = comparisons[comparisons['window'] == window]
    subset = subset[subset['lineages'] == 'ppn-ptr']
    minimum = subset['divergence'].min()
    mean = subset['divergence'].mean()
    maximum = subset['divergence'].max()
    variance = subset['divergence'].var()
    stats.append(subset['window'].iloc[0])
    stats.append(minimum)
    stats.append(mean)
    stats.append(maximum)
    stats.append(variance)
    
    return stats

In [157]:
#ppn_ptr_window_stats = [get_ppn_ptr_window_stats(window) for window in ppn_ptr_windows['window']]

In [158]:
#ppn_ptr_window_stats = pd.DataFrame(ppn_ptr_window_stats, columns = ['window', 'minimum', 'mean', 'maximum', 'variance'])
#ppn_ptr_window_stats.head(5)

Save this dataframe.

In [159]:
#ppn_ptr_window_stats.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_window_stats.txt', sep = '\t', header = True, index = False)

Read in dataframe once it's been created.

In [160]:
ppn_ptr_window_stats = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_window_stats.txt', sep = '\t', header = 0)
ppn_ptr_window_stats.head(5)

,window,minimum,mean,maximum,variance
0,chr10_3670016,0.008749,0.029608,0.096248,2.549325e-04
1,chr10_4194304,0.001334,0.003769,0.012365,3.552572e-06
2,chr10_23592960,0.003272,0.019802,0.055726,9.082293e-05
3,chr10_28311552,0.000317,0.001306,0.004317,4.994337e-07
4,chr10_30932992,0.003739,0.011764,0.024301,1.398396e-05


The bonobo-chimpanzee clustering windows include quite a few with minimum divergence. Let's get a count of the number of windows using different thresholds.

In [161]:
len(ppn_ptr_window_stats[ppn_ptr_window_stats['minimum'] > 0.01])

89

In [162]:
len(ppn_ptr_window_stats[ppn_ptr_window_stats['minimum'] > 0.02])

56

In [163]:
len(ppn_ptr_window_stats[ppn_ptr_window_stats['minimum'] > 0.03])

39

In [164]:
len(ppn_ptr_window_stats[ppn_ptr_window_stats['minimum'] > 0.04])

27

In [165]:
len(ppn_ptr_window_stats[ppn_ptr_window_stats['minimum'] > 0.05])

20

Let's examine the divergence for the example in Figure 3A.

In [166]:
comparisons[(comparisons['window'] == 'chr5_16252928') & (comparisons['ind1'] == 'Clint') & (comparisons['ind2'] == 'Hortense')]

,ind1,ind2,lineages,chr,window_start,window,mse,spearman,divergence,seq_diff
3323824,Clint,Hortense,ppn-ptr,chr5,16252928,chr5_16252928,0.018395,0.96789,0.03211,4441


In [167]:
comparisons[(comparisons['window'] == 'chr5_16252928') & (comparisons['lineages'] == 'ppn-ptr')]['divergence'].min()

0.02491708517

In [168]:
comparisons[(comparisons['window'] == 'chr5_16252928') & (comparisons['lineages'] == 'ppn-ptr')]['divergence'].max()

0.03670547712

Let's collect a random sample of windows at the lower thresholds to visualize map differences at those cutoffs.

In [169]:
comparisons[(comparisons['window'] == 'chr21_24117248') & (comparisons['lineages'] == 'ppn-ptr')].sort_values(by = 'divergence').iloc[0]

ind1                      Bono
ind2                      Koto
lineages               ppn-ptr
chr                      chr21
window_start          24117248
window          chr21_24117248
mse                   0.000601
spearman              0.998292
divergence            0.001708
seq_diff                  4174
Name: 1892425, dtype: object

In [170]:
ppn_ptr_window_stats[ppn_ptr_window_stats['window'] == 'chr21_24117248']

,window,minimum,mean,maximum,variance
139,chr21_24117248,0.001708,0.011038,0.034644,0.00003


In [171]:
comparisons[(comparisons['window'] == 'chr3_88604672') & (comparisons['lineages'] == 'ppn-ptr')].sort_values(by = 'divergence').iloc[0]

ind1                  Kidongo
ind2                   Kosana
lineages              ppn-ptr
chr                      chr3
window_start         88604672
window          chr3_88604672
mse                  0.000306
spearman             0.994241
divergence           0.005759
seq_diff                 4150
Name: 5272682, dtype: object

In [172]:
ppn_ptr_window_stats[ppn_ptr_window_stats['window'] == 'chr3_88604672']

,window,minimum,mean,maximum,variance
187,chr3_88604672,0.005759,0.01364,0.030025,0.000016


In [173]:
comparisons[(comparisons['window'] == 'chr7_137887744') & (comparisons['lineages'] == 'ppn-ptr')].sort_values(by = 'divergence').iloc[0]

ind1                   Hermien
ind2                     Linda
lineages               ppn-ptr
chr                       chr7
window_start         137887744
window          chr7_137887744
mse                   0.003671
spearman              0.991005
divergence            0.008995
seq_diff                  3731
Name: 4570328, dtype: object

In [174]:
ppn_ptr_window_stats[ppn_ptr_window_stats['window'] == 'chr7_137887744']

,window,minimum,mean,maximum,variance
280,chr7_137887744,0.008995,0.015233,0.031131,0.000013


In [175]:
comparisons[(comparisons['window'] == 'chr9_62914560') & (comparisons['lineages'] == 'ppn-ptr')].sort_values(by = 'divergence').iloc[0]

ind1                   Dzeeta
ind2                     Luky
lineages              ppn-ptr
chr                      chr9
window_start         62914560
window          chr9_62914560
mse                   0.00152
spearman             0.988534
divergence           0.011466
seq_diff                 3190
Name: 4158497, dtype: object

In [176]:
ppn_ptr_window_stats[ppn_ptr_window_stats['window'] == 'chr9_62914560']

,window,minimum,mean,maximum,variance
304,chr9_62914560,0.011466,0.019278,0.035595,0.000013


Let's define "bonobo-chimpanzee divergent windows" as those with a minimum interspecific 3d divergence of 0.01.

In [177]:
ppn_ptr_divergent_windows = ppn_ptr_window_stats[ppn_ptr_window_stats['minimum'] >= 0.01]['window'].to_frame('window')
ppn_ptr_divergent_windows.head(5)

,window
9,chr10_78643200
13,chr10_87556096
20,chr11_22544384
28,chr11_77070336
29,chr11_77594624


Create and saving a BED file of the divergent bonobo-chimpanzee windows.

In [178]:
ppn_ptr_divergent_windows_BED = pd.DataFrame()
ppn_ptr_divergent_windows_BED['window_split'] = ppn_ptr_divergent_windows['window']
ppn_ptr_divergent_windows_BED = ppn_ptr_divergent_windows_BED['window_split'].str.split('_', expand=True)
ppn_ptr_divergent_windows_BED.rename(columns = {0:'chr', 1:'window_start'}, inplace = True)
ppn_ptr_divergent_windows_BED['window_start'] = ppn_ptr_divergent_windows_BED['window_start'].astype(int)
ppn_ptr_divergent_windows_BED['window_end'] = ppn_ptr_divergent_windows_BED['window_start'] + 1048576
ppn_ptr_divergent_windows_BED = ppn_ptr_divergent_windows_BED.sort_values(by = ['chr','window_start'])
ppn_ptr_divergent_windows_BED.head(5)

,chr,window_start,window_end
115,chr1,126353408,127401984
116,chr1,153092096,154140672
117,chr1,153616384,154664960
119,chr1,173015040,174063616
9,chr10,78643200,79691776


In [179]:
ppn_ptr_divergent_windows_BED.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_windows.bed', sep = '\t', header = False, index = False)

How are these distributed among the chromosomes?

In [180]:
ppn_ptr_divergent_windows_BED.groupby(['chr']).size().to_frame('N')

,N
chr,
chr1,4
chr10,2
chr11,3
chr12,2
chr13,7
chr14,4
chr15,3
chr16,1
chr18,2


How many of these loci are unique (i.e., non-overlapping)? First, create a pybedtools object (we'll use this later) and then perform a merge.

In [181]:
ppn_ptr_divergent_windows_pbtBED = pybedtools.BedTool().from_dataframe(ppn_ptr_divergent_windows_BED).sort()
ppn_ptr_divergent_windows_pbtBED.head()

chr1	126353408	127401984
 chr1	153092096	154140672
 chr1	153616384	154664960
 chr1	173015040	174063616
 chr10	78643200	79691776
 chr10	87556096	88604672
 chr11	22544384	23592960
 chr11	77070336	78118912
 chr11	77594624	78643200
 chr12	37748736	38797312
 

In [182]:
len(ppn_ptr_divergent_windows_pbtBED.merge())

71

### Genes Total and Genes per Window <a class = 'anchor' id = 'genestotalgenesperwindow'></a>

Let's intersect this object with genes in the panTro6 genome.

In [183]:
genes_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/annotations/panTro6_genes.bed')
genes_pbtBED.head(5)

chr1	344	12299	NM_001280424.1	INTS11
 chr1	12400	16513	XM_016952033.2	CPTP
 chr1	17887	23426	XM_003307748.4	TAS1R3
 chr1	22943	36942	XM_016958290.2	DVL1
 chr1	39619	45507	NM_001280245.1	MXRA8
 

In [184]:
ppn_ptr_divergent_windows_genes_intersect = ppn_ptr_divergent_windows_pbtBED.intersect(genes_pbtBED, wa = True, wb = True).to_dataframe(names=['window_chr','window_start','window_end','gene_chr','gene_start','gene_end','gene_transcript','gene'])
ppn_ptr_divergent_windows_genes_intersect.head(5)

,window_chr,window_start,window_end,gene_chr,gene_start,gene_end,gene_transcript,gene
0,chr1,126353408,127401984,chr1,126517594,126522374,XM_009431515.3,MRPL9
1,chr1,126353408,127401984,chr1,126524628,126529347,NM_001172571.2,OAZ3
2,chr1,126353408,127401984,chr1,126558327,126563506,XM_524871.3,LINGO4
3,chr1,126353408,127401984,chr1,126595864,126597165,XM_016927235.2,C2CD4D
4,chr1,126353408,127401984,chr1,126603960,126612050,XM_016927244.1,THEM5


In [185]:
len(ppn_ptr_divergent_windows_genes_intersect)

454

Let's get a list of genes in these windows.

In [186]:
ppn_ptr_divergent_windows_genes = ppn_ptr_divergent_windows_genes_intersect['gene'].drop_duplicates().sort_values()
ppn_ptr_divergent_windows_genes.head(5)

43         ABL2
415         ABO
436        ACE2
346      ADAM22
420    ADAMTS13
Name: gene, dtype: object

In [187]:
len(ppn_ptr_divergent_windows_genes)

431

In [188]:
ppn_ptr_divergent_windows_genes_set = set(ppn_ptr_divergent_windows_genes)

In [189]:
len(ppn_ptr_divergent_windows_genes_set)

431

We will save this file to look for gene enrichment of particular phenotypes.

In [190]:
ppn_ptr_divergent_windows_genes.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/phenotype_enrichment/data/ppn_ptr_divergent_windows_genes.txt', sep = '\t', header = False, index = False)

Let's run another intersection and count the number of genes in each divergent window.

In [191]:
ppn_ptr_divergent_windows_genes_count_intersect = ppn_ptr_divergent_windows_pbtBED.intersect(genes_pbtBED, c = True).to_dataframe(names=['window_chr','window_start','window_end','gene_count'])
ppn_ptr_divergent_windows_genes_count_intersect.head(5)

,window_chr,window_start,window_end,gene_count
0,chr1,126353408,127401984,28
1,chr1,153092096,154140672,7
2,chr1,153616384,154664960,10
3,chr1,173015040,174063616,5
4,chr10,78643200,79691776,1


Save this dataframe for visualization.

In [192]:
ppn_ptr_divergent_windows_genes_count_intersect.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_windows_genes_counts.txt', sep = '\t', header = True, index = False)

In [193]:
ppn_ptr_divergent_windows_genes_count_intersect['gene_count'].min()

0

In [194]:
ppn_ptr_divergent_windows_genes_count_intersect['gene_count'].max()

36

In [195]:
ppn_ptr_divergent_windows_genes_count_intersect['gene_count'].mean()

5.101123595505618

Let's take a look at the distribution of gene counts.

In [196]:
ppn_ptr_divergent_windows_genes_count_intersect.groupby(['gene_count']).size().to_frame('N')

,N
gene_count,
0,17
1,18
2,13
3,8
4,4
5,4
6,2
7,3
8,4


We should compare these to the genome-wide background.

In [197]:
all_windows_genes_count_intersect = windows_pbtBED.intersect(genes_pbtBED, c = True).to_dataframe(names=['window_chr','window_start','window_end','gene_count'])
all_windows_genes_count_intersect.head(5)

,window_chr,window_start,window_end,gene_count
0,chr1,1048576,2097152,14
1,chr1,1572864,2621440,13
2,chr1,2097152,3145728,10
3,chr1,2621440,3670016,1
4,chr1,3145728,4194304,2


Save this dataframe for visualization.

In [198]:
all_windows_genes_count_intersect.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/all_windows_genes_count_intersect.txt', sep = '\t', header = True, index = False)

In [199]:
all_windows_genes_count_intersect['gene_count'].min()

0

In [200]:
all_windows_genes_count_intersect['gene_count'].max()

67

In [201]:
all_windows_genes_count_intersect['gene_count'].mean()

7.511764705882353

In [202]:
mannwhitneyu(all_windows_genes_count_intersect['gene_count'], ppn_ptr_divergent_windows_genes_count_intersect['gene_count'])

MannwhitneyuResult(statistic=252261.5, pvalue=4.590052657934087e-06)

What about the overall sum of genes? Let's load in the randomization results where we sampled 89 random windows and summed the number of genes in all windows 10,000 times.

In [203]:
ppn_ptr_windows_n_genes_randomization = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_windows_n_genes_randomization.txt', sep = '\t', names = ['N'])
ppn_ptr_windows_n_genes_randomization.head(5)

,N
0,705
1,685
2,571
3,745
4,629


In [204]:
len(ppn_ptr_windows_n_genes_randomization)

10000

In [205]:
ppn_ptr_windows_n_genes_randomization['N'].min()

418

In [206]:
ppn_ptr_windows_n_genes_randomization['N'].max()

967

In [207]:
ppn_ptr_windows_n_genes_randomization['N'].mean()

659.6825

In [208]:
len(ppn_ptr_windows_n_genes_randomization[ppn_ptr_windows_n_genes_randomization['N'] <= 431])

2

In [209]:
2/10000

0.0002

In [210]:
431/659.6825

0.6533446013802094

What if we rank the most divergent bonobo-chimpanzee windows? Is there an association between divergence score and number of genes?

In [211]:
ppn_ptr_divergent_window_stats = ppn_ptr_window_stats[ppn_ptr_window_stats['window'].isin(ppn_ptr_divergent_windows['window'])]
len(ppn_ptr_divergent_window_stats)

89

In [212]:
ppn_ptr_divergent_windows_genes_count_intersect['window'] = ppn_ptr_divergent_windows_genes_count_intersect['window_chr'] + '_' + ppn_ptr_divergent_windows_genes_count_intersect['window_start'].astype(str)
ppn_ptr_divergent_windows_genes_count_intersect.head(5)

,window_chr,window_start,window_end,gene_count,window
0,chr1,126353408,127401984,28,chr1_126353408
1,chr1,153092096,154140672,7,chr1_153092096
2,chr1,153616384,154664960,10,chr1_153616384
3,chr1,173015040,174063616,5,chr1_173015040
4,chr10,78643200,79691776,1,chr10_78643200


In [213]:
ppn_ptr_divergent_window_stats_gene_count = pd.merge(ppn_ptr_divergent_window_stats, ppn_ptr_divergent_windows_genes_count_intersect[['window','gene_count']], on = 'window', how = 'inner')
ppn_ptr_divergent_window_stats_gene_count.head(5)

,window,minimum,mean,maximum,variance,gene_count
0,chr10_78643200,0.011702,0.031134,0.052345,0.000043,1
1,chr10_87556096,0.026767,0.033005,0.040717,0.000005,8
2,chr11_22544384,0.014435,0.024658,0.050180,0.000037,3
3,chr11_77070336,0.023994,0.037768,0.057039,0.000029,0
4,chr11_77594624,0.050357,0.076725,0.122029,0.000142,2


In [214]:
ppn_ptr_divergent_window_stats_gene_count.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_window_stats_gene_count.txt', sep = '\t', header = True, index = False)

In [215]:
rho, p = spearmanr(ppn_ptr_divergent_window_stats_gene_count['minimum'], ppn_ptr_divergent_window_stats_gene_count['gene_count'])
print(rho,p)

-0.3673811631053057 0.00039775579289958807


In [216]:
rho, p = spearmanr(ppn_ptr_divergent_window_stats_gene_count['mean'], ppn_ptr_divergent_window_stats_gene_count['gene_count'])
print(rho,p)

-0.49078994548019744 1.0457737105949218e-06


In [217]:
rho, p = spearmanr(ppn_ptr_divergent_window_stats_gene_count['maximum'], ppn_ptr_divergent_window_stats_gene_count['gene_count'])
print(rho,p)

-0.5060559479527063 4.2132088900800013e-07


### phastCons Elements <a class = 'anchor' id = 'phastconselements'></a>

Let's also consider conservation in these windows by the proportion of phastCons conserved elements in bonobo-chimpanzee divergent windows. Run a script to download and liftOver the conserved elements from hg38 to panTro6 and filter for elements with LOD $\geq$ 500.

In [218]:
phastCons_elements_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/phastCons/phastConsElements30way_panTro6.bed')
phastCons_elements_pbtBED.head(5)

chr1	613	772	552
 chr1	6190	6427	612
 chr1	36658	36857	586
 chr1	76342	76520	529
 chr1	80960	81083	500
 

In [219]:
len(phastCons_elements_pbtBED)

235405

In [220]:
phastCons_elements_ppn_ptr_divergent_windows_intersect = ppn_ptr_divergent_windows_pbtBED.intersect(phastCons_elements_pbtBED, wao = True).to_dataframe(names=['window_chr','window_start','window_end','element_chr','element_start','element_end','element_score','bp_overlap']) 
phastCons_elements_ppn_ptr_divergent_windows_intersect.head(5)

,window_chr,window_start,window_end,element_chr,element_start,element_end,element_score,bp_overlap
0,chr1,126353408,127401984,chr1,126367862,126368248,668,386
1,chr1,126353408,127401984,chr1,126371694,126371855,508,161
2,chr1,126353408,127401984,chr1,126394894,126395348,696,454
3,chr1,126353408,127401984,chr1,126411938,126412184,606,246
4,chr1,126353408,127401984,chr1,126414434,126414614,515,180


In [221]:
phastCons_elements_ppn_ptr_divergent_windows = phastCons_elements_ppn_ptr_divergent_windows_intersect.copy()
phastCons_elements_ppn_ptr_divergent_windows['window'] = phastCons_elements_ppn_ptr_divergent_windows['window_chr'] + '_' + phastCons_elements_ppn_ptr_divergent_windows['window_start'].astype(str)
phastCons_elements_ppn_ptr_divergent_windows = phastCons_elements_ppn_ptr_divergent_windows.drop(columns = ['window_chr','window_start','window_end','element_chr','element_start','element_end','element_score'])
phastCons_elements_ppn_ptr_divergent_windows = phastCons_elements_ppn_ptr_divergent_windows[['window','bp_overlap']]
phastCons_elements_ppn_ptr_divergent_windows = phastCons_elements_ppn_ptr_divergent_windows.groupby(['window']).sum('bp_overlap').reset_index().sort_values(['window'], ascending = True)
phastCons_elements_ppn_ptr_divergent_windows['prop'] = phastCons_elements_ppn_ptr_divergent_windows['bp_overlap']/1048576
phastCons_elements_ppn_ptr_divergent_windows.head(5)

,window,bp_overlap,prop
0,chr10_78643200,4816,0.004593
1,chr10_87556096,34393,0.032800
2,chr11_22544384,484,0.000462
3,chr11_77070336,6701,0.006391
4,chr11_77594624,12147,0.011584


Let's compare this to the genome-wide background.

In [222]:
phastCons_elements_all_windows_intersect = windows_pbtBED.intersect(phastCons_elements_pbtBED, wao = True).to_dataframe(names=['window_chr','window_start','window_end','element_chr','element_start','element_end','element_score','bp_overlap']) 
phastCons_elements_all_windows_intersect.head(5)

/wynton/home/capra/cbrand/miniconda3/envs/jupyter/lib/python3.10/site-packages/pybedtools/bedtool.py:3714: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pandas.read_csv(self.fn, *args, sep="\t", **kwargs)


,window_chr,window_start,window_end,element_chr,element_start,element_end,element_score,bp_overlap
0,chr1,1048576,2097152,chr1,1086782,1086949,545,167
1,chr1,1048576,2097152,chr1,1087296,1087406,534,110
2,chr1,1048576,2097152,chr1,1185992,1186177,528,185
3,chr1,1048576,2097152,chr1,1200100,1200249,570,149
4,chr1,1048576,2097152,chr1,1200111,1200248,585,137


In [223]:
phastCons_elements_all_windows = phastCons_elements_all_windows_intersect.copy()
phastCons_elements_all_windows['window'] = phastCons_elements_all_windows['window_chr'] + '_' + phastCons_elements_all_windows['window_start'].astype(str)
phastCons_elements_all_windows = phastCons_elements_all_windows.drop(columns = ['window_chr','window_start','window_end','element_chr','element_start','element_end','element_score'])
phastCons_elements_all_windows = phastCons_elements_all_windows[['window','bp_overlap']]
phastCons_elements_all_windows = phastCons_elements_all_windows.groupby(['window']).sum('bp_overlap').reset_index().sort_values(['window'], ascending = True)
phastCons_elements_all_windows['prop'] = phastCons_elements_all_windows['bp_overlap']/1048576
phastCons_elements_all_windows.head(5)

,window,bp_overlap,prop
0,chr10_100139008,26802,0.025560
1,chr10_100663296,19669,0.018758
2,chr10_101187584,11165,0.010648
3,chr10_101711872,16447,0.015685
4,chr10_102236160,16215,0.015464


In [224]:
len(phastCons_elements_all_windows)

4420

In [225]:
phastCons_elements_all_windows['prop'].mean()

0.020641775044920218

How many bonobo-chimpanzee divergent windows are below and above this genome-wide mean?

In [226]:
len(phastCons_elements_ppn_ptr_divergent_windows[phastCons_elements_ppn_ptr_divergent_windows['prop'] < 0.020641775044920218])

66

In [227]:
len(phastCons_elements_ppn_ptr_divergent_windows[phastCons_elements_ppn_ptr_divergent_windows['prop'] > 0.020641775044920218])

23

Let's map on maximum divergence per window and examine the relationship between proportion of conserved elements and divergence.

In [228]:
ppn_ptr_divergent_window_maxes_dict = dict(zip(ppn_ptr_window_stats['window'], ppn_ptr_window_stats['maximum']))
phastCons_elements_ppn_ptr_divergent_windows['maximum'] = phastCons_elements_ppn_ptr_divergent_windows['window'].map(ppn_ptr_divergent_window_maxes_dict)
phastCons_elements_ppn_ptr_divergent_windows.head(5)

,window,bp_overlap,prop,maximum
0,chr10_78643200,4816,0.004593,0.052345
1,chr10_87556096,34393,0.032800,0.040717
2,chr11_22544384,484,0.000462,0.050180
3,chr11_77070336,6701,0.006391,0.057039
4,chr11_77594624,12147,0.011584,0.122029


In [229]:
len(phastCons_elements_ppn_ptr_divergent_windows)

89

In [230]:
rho, p = spearmanr(phastCons_elements_ppn_ptr_divergent_windows['maximum'], phastCons_elements_ppn_ptr_divergent_windows['prop'])
print(rho,p)

-0.1821416411304052 0.08757112628834791


Peak at the windows with the highest divergence.

In [231]:
phastCons_elements_ppn_ptr_divergent_windows.sort_values('prop', ascending = False).head(5)

,window,bp_overlap,prop,maximum
12,chr13_52428800,89429,0.085286,0.248103
26,chr19_51380224,72334,0.068983,0.103107
39,chr2B_42991616,54387,0.051867,0.020830
46,chr3_144703488,50137,0.047814,0.112864
75,chr8_113770496,39926,0.038076,0.159110


In [232]:
phastCons_elements_ppn_ptr_divergent_windows.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/phastCons/phastCons_elements_ppn_ptr_divergent_windows.txt', sep = '\t', header = True, index = False)

## Phenotype Enrichment <a class = 'anchor' id = 'phenotypeenrichment'></a>

Let's examine the results of our phenotype enrichment pipeline.

In [233]:
fdr_table = []

In [234]:
def reportFDRcorrectedPthreshold(set_name, ontology, q_value_threshold, resolution=0.0001, minStart=0):
    fdr_empiric = pd.read_csv(f'/wynton/group/capra/projects/pan_3d_genome/data/phenotype_enrichment/empiric_FDR/{set_name}_{ontology}_empiric_FDR.txt', sep = '\t', header = None, index_col = 0)
    obs = pd.read_csv(f'/wynton/group/capra/projects/pan_3d_genome/data/phenotype_enrichment/enrichment/{set_name}_{ontology}_enrichment.txt', sep = '\t')

    fdr_threshold = []
    for i in np.arange(minStart,0.05,resolution):
        
        observed_positive = sum(obs['p_value'] <= i)
        average_false_positive = (fdr_empiric <= i).sum().mean()
        q = average_false_positive/observed_positive
        fdr_threshold.append([set_name, ontology, q_value_threshold, i, observed_positive, average_false_positive, q])
        
        if (q != np.inf) & (q > q_value_threshold):
            break
    
    threshold = fdr_threshold[-2]
    fdr_table.append(threshold)
    #fdr_threshold = pd.DataFrame(fdr_threshold, columns = ['pval_threshold','obsPos','avgFalsePos','q'])
    #return fdr_threshold.tail(2).head(1)

In [235]:
combinations = [(set_name,ontology,q_value_threshold) for set_name in ['ppn_ptr_clustering'] for ontology in ['BP','GWAS','HPO','MP'] for q_value_threshold in [0.05,0.1]]

In [236]:
[reportFDRcorrectedPthreshold(set_name, ontology, q_value_threshold) for set_name, ontology, q_value_threshold in combinations]

/scratch/cbrand/ipykernel_2007303/1065567570.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_2007303/1065567570.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_2007303/1065567570.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_2007303/1065567570.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_2007303/1065567570.py:10: RuntimeWarning: invalid value encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_2007303/1065567570.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  q = average_false_positive/observed_positive
/scratch/cbrand/ipykernel_2007303/106556757

[None, None, None, None, None, None, None, None]

In [237]:
fdr_table = pd.DataFrame(fdr_table, columns = ['set', 'ontology', 'q_value_threshold', 'p_value_threshold','observed_positive','average_false_positive','q'])
fdr_table

,set,ontology,q_value_threshold,p_value_threshold,observed_positive,average_false_positive,q
0,ppn_ptr_clustering,BP,0.05,0.0078,0,6.4675,inf
1,ppn_ptr_clustering,BP,0.10,0.0078,0,6.4675,inf
2,ppn_ptr_clustering,GWAS,0.05,0.0077,0,1.7036,inf
3,ppn_ptr_clustering,GWAS,0.10,0.0077,0,1.7036,inf
4,ppn_ptr_clustering,HPO,0.05,0.0094,0,2.4849,inf
5,ppn_ptr_clustering,HPO,0.10,0.0094,0,2.4849,inf
6,ppn_ptr_clustering,MP,0.05,0.0019,0,1.1145,inf
7,ppn_ptr_clustering,MP,0.10,0.0019,0,1.1145,inf


### *In Silico* Mutagenesis for Bonobo-Specific Variants <a class = 'anchor' id = 'ppninsilicomutagenesis'></a>

What individual variants are driving divergence in bonobo-chimpanzee divergent windows? Let's examine the results of *in silico* mutagenesis for bonobo-specific variants.

In [238]:
ppn_ptr_divergent_window_variants = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/in_silico_mutagenesis/ppn_specific_variants_in_ppn_ptr_divergent_windows_in_silico_mutagenesis.txt', sep = '\t', header = 0)
ppn_ptr_divergent_window_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence
0,chr1,126353856,C,T,chr1_126353408,6.057105e-09,2.141901e-08
1,chr1,126354684,C,T,chr1_126353408,5.636190e-10,2.657346e-09
2,chr1,126357977,T,C,chr1_126353408,1.889927e-09,6.916147e-09
3,chr1,126358737,C,T,chr1_126353408,1.360100e-09,5.963432e-09
4,chr1,126361540,G,T,chr1_126353408,4.040863e-10,1.929300e-09


In [239]:
len(ppn_ptr_divergent_window_variants[['chr','pos','ref','alt']].drop_duplicates())

115191

In [240]:
len(ppn_ptr_divergent_window_variants)

127075

How many windows are represented by these variants?

In [241]:
len(ppn_ptr_divergent_window_variants['window'].unique())

89

To identify 3d-modifying variants, let's map the lowest bonobo-chimpanzee divergence score to the above dataframe.

In [242]:
ppn_ptr_divergent_window_mins_dict = dict(zip(ppn_ptr_window_stats['window'], ppn_ptr_window_stats['minimum']))

In [243]:
ppn_ptr_divergent_window_variants['min'] = ppn_ptr_divergent_window_variants['window'].map(ppn_ptr_divergent_window_mins_dict)

In [244]:
ppn_ptr_divergent_window_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence,min
0,chr1,126353856,C,T,chr1_126353408,6.057105e-09,2.141901e-08,0.029323
1,chr1,126354684,C,T,chr1_126353408,5.636190e-10,2.657346e-09,0.029323
2,chr1,126357977,T,C,chr1_126353408,1.889927e-09,6.916147e-09,0.029323
3,chr1,126358737,C,T,chr1_126353408,1.360100e-09,5.963432e-09,0.029323
4,chr1,126361540,G,T,chr1_126353408,4.040863e-10,1.929300e-09,0.029323


Now let's identify variants with a measured effect $\geq$ than the lowest divergence score among bonobo-chimpanzee pairwise comparisons for that window.

In [245]:
ppn_ptr_divergent_window_3d_modifying_variants = ppn_ptr_divergent_window_variants[ppn_ptr_divergent_window_variants['divergence'] >= ppn_ptr_divergent_window_variants['min']]
ppn_ptr_divergent_window_3d_modifying_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence,min
651,chr1,126940560,C,G,chr1_126353408,0.011125,0.036647,0.029323
2146,chr1,153901388,A,T,chr1_153616384,0.005189,0.017142,0.016506
7964,chr10,88057593,G,C,chr10_87556096,0.018322,0.031364,0.026767
7965,chr10,88057595,G,T,chr10_87556096,0.019119,0.032893,0.026767
12773,chr11,77997697,G,A,chr11_77070336,0.002093,0.025199,0.023994


In [246]:
len(ppn_ptr_divergent_window_3d_modifying_variants)

61

In [247]:
len(ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos','ref','alt']].drop_duplicates())

51

Let's save a version of this dataframe now. We will create another with additional information later.

In [248]:
ppn_ptr_divergent_window_3d_modifying_variants.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_window_3d_modifying_variants.txt', sep = '\t', header = False, index = False)

The presence of duplicates indicates the presence of following scenarios: 1) multiple 3d-modifying variants per window or 2) a 3d-modifying variant impacting adjacent windows. Let's assess.

In [249]:
multi_window_ppn_ptr_divergent_window_3d_modifying_variants = ppn_ptr_divergent_window_3d_modifying_variants[ppn_ptr_divergent_window_3d_modifying_variants.duplicated(['chr','pos'], keep = False)]
multi_window_ppn_ptr_divergent_window_3d_modifying_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence,min
12773,chr11,77997697,G,A,chr11_77070336,0.002093,0.025199,0.023994
12774,chr11,77997697,G,A,chr11_77594624,0.007758,0.069116,0.050357
30827,chr14,45793476,G,T,chr14_45088768,0.011070,0.030854,0.020095
30828,chr14,45793476,G,T,chr14_45613056,0.007928,0.033563,0.019686
44464,chr19,26559939,A,T,chr19_25690112,0.006024,0.039807,0.029334


In [250]:
len(multi_window_ppn_ptr_divergent_window_3d_modifying_variants)/2

10.0

10 windows have a 3d-modifying variant that impacts both windows. What about windows with multiple variants?

In [251]:
ppn_ptr_divergent_window_3d_modifying_variants_n_variants_per_window = ppn_ptr_divergent_window_3d_modifying_variants.groupby(['window']).size().to_frame('N')
ppn_ptr_divergent_window_3d_modifying_variants_n_variants_per_window[ppn_ptr_divergent_window_3d_modifying_variants_n_variants_per_window['N'] > 1]

,N
window,
chr10_87556096,2
chr4_113770496,2


In [252]:
ppn_ptr_divergent_window_3d_modifying_variants[ppn_ptr_divergent_window_3d_modifying_variants['window'] == 'chr10_87556096']

,chr,pos,ref,alt,window,mse,divergence,min
7964,chr10,88057593,G,C,chr10_87556096,0.018322,0.031364,0.026767
7965,chr10,88057595,G,T,chr10_87556096,0.019119,0.032893,0.026767


In [253]:
ppn_ptr_divergent_window_3d_modifying_variants[ppn_ptr_divergent_window_3d_modifying_variants['window'] == 'chr4_113770496']

,chr,pos,ref,alt,window,mse,divergence,min
88832,chr4,114115871,G,A,chr4_113770496,0.000826,0.038260,0.029619
88833,chr4,114115875,C,T,chr4_113770496,0.000969,0.045659,0.029619


How many windows are covered by 3d-modifying variants?

In [254]:
len(ppn_ptr_divergent_window_3d_modifying_variants['window'].unique())

59

In [255]:
59/89

0.6629213483146067

In [256]:
comparisons[(comparisons['window'] == 'chr7_83886080') & (comparisons['lineages'] == 'ppn-ptr')]['divergence'].min()

0.036591365416

In [257]:
comparisons[(comparisons['window'] == 'chr7_83886080') & (comparisons['lineages'] == 'ppn-ptr')]['divergence'].max()

0.0699732955899999

Let's create a quick BED file of these windows.

In [258]:
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants['window'].str.split('_', expand = True).rename(columns = {0:'chr', 1:'start'})
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants_BED.drop_duplicates()
ppn_ptr_divergent_window_3d_modifying_variants_BED['start'] = ppn_ptr_divergent_window_3d_modifying_variants_BED['start'].astype(int)
ppn_ptr_divergent_window_3d_modifying_variants_BED['end'] = ppn_ptr_divergent_window_3d_modifying_variants_BED['start'] + 1048576
ppn_ptr_divergent_window_3d_modifying_variants_BED.head(5)

,chr,start,end
651,chr1,126353408,127401984
2146,chr1,153616384,154664960
7964,chr10,87556096,88604672
12773,chr11,77070336,78118912
12774,chr11,77594624,78643200


In [259]:
len(ppn_ptr_divergent_window_3d_modifying_variants_BED)

59

Are there differences in the mutation frequency of these variants?

In [260]:
ppn_ptr_divergent_window_3d_modifying_unique_variants = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos','ref','alt']].drop_duplicates()

In [261]:
ppn_ptr_divergent_window_3d_modifying_unique_variant_counts = ppn_ptr_divergent_window_3d_modifying_unique_variants.groupby(['ref','alt']).size().to_frame('N').reset_index()
ppn_ptr_divergent_window_3d_modifying_unique_variant_counts['prop'] = ppn_ptr_divergent_window_3d_modifying_unique_variant_counts['N']/ppn_ptr_divergent_window_3d_modifying_unique_variant_counts['N'].sum()
ppn_ptr_divergent_window_3d_modifying_unique_variant_counts

,ref,alt,N,prop
0,A,C,5,0.098039
1,A,G,3,0.058824
2,A,T,2,0.039216
3,C,A,1,0.019608
4,C,G,3,0.058824
5,C,T,14,0.274510
6,G,A,8,0.156863
7,G,C,5,0.098039
8,G,T,4,0.078431
9,T,C,5,0.098039


Let's manually add the missing rows to this dataframe for visualization and save.

In [262]:
new_rows = [{'ref': 'A', 'alt': 'A', 'N': np.nan, 'prop': np.nan},
            {'ref': 'C', 'alt': 'C', 'N': np.nan, 'prop': np.nan},
            {'ref': 'G', 'alt': 'G', 'N': np.nan, 'prop': np.nan},
            {'ref': 'T', 'alt': 'A', 'N': 0, 'prop': 0},
            {'ref': 'T', 'alt': 'T', 'N': np.nan, 'prop': np.nan}]

new_df = pd.DataFrame(new_rows)

In [263]:
ppn_ptr_divergent_window_3d_modifying_unique_variant_counts = pd.concat([ppn_ptr_divergent_window_3d_modifying_unique_variant_counts, new_df], ignore_index=True).sort_values(by = ['ref','alt'], ascending = True)
ppn_ptr_divergent_window_3d_modifying_unique_variant_counts

,ref,alt,N,prop
11,A,A,NaN,NaN
0,A,C,5.0,0.098039
1,A,G,3.0,0.058824
2,A,T,2.0,0.039216
3,C,A,1.0,0.019608
12,C,C,NaN,NaN
4,C,G,3.0,0.058824
5,C,T,14.0,0.274510
6,G,A,8.0,0.156863
7,G,C,5.0,0.098039


In [264]:
ppn_ptr_divergent_window_3d_modifying_unique_variant_counts.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_window_3d_modifying_unique_variant_counts.txt', sep = '\t', header = True, index = False)

How frequent are transitions?

In [265]:
(3+14+8+5)/59

0.5084745762711864

There are quite a few C > T mutations. How many of these are CpGs? Run the CpG retrieval script and then read in the output.

In [266]:
ppn_ptr_divergent_window_3d_modifying_variants_dinucleotides = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_window_3d_modifying_variants_dinucleotides.txt', sep = '\t', header = 0)
ppn_ptr_divergent_window_3d_modifying_variants_dinucleotides.head(5)

,chr,pos,window,ref,alt,CpG
0,chr1,126940560,chr1_126353408,C,G,CT
1,chr1,153901388,chr1_153616384,A,T,NaN
2,chr10,88057593,chr10_87556096,G,C,NaN
3,chr10,88057595,chr10_87556096,G,T,NaN
4,chr11,77997697,chr11_77070336,G,A,NaN


In [267]:
ppn_ptr_divergent_window_3d_modifying_variants_with_CpG_counts = ppn_ptr_divergent_window_3d_modifying_variants_dinucleotides[ppn_ptr_divergent_window_3d_modifying_variants_dinucleotides['ref'] == 'C']
ppn_ptr_divergent_window_3d_modifying_variants_with_CpG_counts = ppn_ptr_divergent_window_3d_modifying_variants_with_CpG_counts[['chr','pos','ref','alt','CpG']].drop_duplicates()
ppn_ptr_divergent_window_3d_modifying_variants_with_CpG_counts = ppn_ptr_divergent_window_3d_modifying_variants_with_CpG_counts.groupby(['CpG']).size().to_frame('N')
ppn_ptr_divergent_window_3d_modifying_variants_with_CpG_counts

,N
CpG,
CA,4
CC,4
CG,3
CT,7


### Ancestral Allele <a class = 'anchor' id = 'ancestralallele'></a>

What is the ancestral allele for 3d-modifying variants? We predict that most should be derived. First, let's send a BED file to a directory with a pipeline that can retrieve ancestral alleles.

In [268]:
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos']].copy()
ppn_ptr_divergent_window_3d_modifying_variants_BED.rename(columns = {'pos':'end'}, inplace = True)
ppn_ptr_divergent_window_3d_modifying_variants_BED['start'] = ppn_ptr_divergent_window_3d_modifying_variants_BED['end'] - 1
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants_BED[['chr','start','end']]
ppn_ptr_divergent_window_3d_modifying_variants_BED.head()

,chr,start,end
651,chr1,126940559,126940560
2146,chr1,153901387,153901388
7964,chr10,88057592,88057593
7965,chr10,88057594,88057595
12773,chr11,77997696,77997697


In [269]:
ppn_ptr_divergent_window_3d_modifying_variants_BED.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ancestral_allele/data/ppn_ptr_divergent_window_3d_modifying_variants.bed', sep = '\t', header = True, index = False)

Now run the pipeline and read in the output.

In [270]:
ppn_ptr_divergent_window_3d_modifying_variants_ancestral_alleles = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ancestral_allele/data/ppn_ptr_divergent_window_3d_modifying_variants_ancestral_allele.txt', sep = '\t', names = ['chr','pos','ancestral_allele'])
ppn_ptr_divergent_window_3d_modifying_variants_ancestral_alleles.head(5)

,chr,pos,ancestral_allele
0,chr1,126940560,C
1,chr1,153901388,A
2,chr10,88057593,G
3,chr10,88057595,G
4,chr11,77997697,G


In [271]:
ppn_ptr_divergent_window_3d_modifying_variants = pd.merge(ppn_ptr_divergent_window_3d_modifying_variants, ppn_ptr_divergent_window_3d_modifying_variants_ancestral_alleles, on = ['chr','pos'])
ppn_ptr_divergent_window_3d_modifying_variants = ppn_ptr_divergent_window_3d_modifying_variants.drop_duplicates()
ppn_ptr_divergent_window_3d_modifying_variants = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos','ref','alt','ancestral_allele','window','mse','divergence','min']]
ppn_ptr_divergent_window_3d_modifying_variants.head(5)

,chr,pos,ref,alt,ancestral_allele,window,mse,divergence,min
0,chr1,126940560,C,G,C,chr1_126353408,0.011125,0.036647,0.029323
1,chr1,153901388,A,T,A,chr1_153616384,0.005189,0.017142,0.016506
2,chr10,88057593,G,C,G,chr10_87556096,0.018322,0.031364,0.026767
3,chr10,88057595,G,T,G,chr10_87556096,0.019119,0.032893,0.026767
4,chr11,77997697,G,A,G,chr11_77070336,0.002093,0.025199,0.023994


In [272]:
len(ppn_ptr_divergent_window_3d_modifying_variants)

61

In [273]:
ppn_ptr_divergent_window_3d_modifying_variants_dedup = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos','ref','alt','ancestral_allele']].drop_duplicates()
ppn_ptr_divergent_window_3d_modifying_variants_dedup.head(5)

,chr,pos,ref,alt,ancestral_allele
0,chr1,126940560,C,G,C
1,chr1,153901388,A,T,A
2,chr10,88057593,G,C,G
3,chr10,88057595,G,T,G
4,chr11,77997697,G,A,G


In [274]:
def alt_ancestral_derived(ppn_ptr_divergent_window_3d_modifying_variants):
    if ppn_ptr_divergent_window_3d_modifying_variants['ancestral_allele'].upper() == ppn_ptr_divergent_window_3d_modifying_variants['alt']:
        return 'ancestral'
    else:
        return 'derived'

ppn_ptr_divergent_window_3d_modifying_variants['alt_ancestral_derived'] = ppn_ptr_divergent_window_3d_modifying_variants.apply(alt_ancestral_derived, axis = 1)

In [275]:
ppn_ptr_divergent_window_3d_modifying_variants = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos','ref','alt','ancestral_allele','alt_ancestral_derived','window','mse','divergence','min']]
ppn_ptr_divergent_window_3d_modifying_variants.head(5)

,chr,pos,ref,alt,ancestral_allele,alt_ancestral_derived,window,mse,divergence,min
0,chr1,126940560,C,G,C,derived,chr1_126353408,0.011125,0.036647,0.029323
1,chr1,153901388,A,T,A,derived,chr1_153616384,0.005189,0.017142,0.016506
2,chr10,88057593,G,C,G,derived,chr10_87556096,0.018322,0.031364,0.026767
3,chr10,88057595,G,T,G,derived,chr10_87556096,0.019119,0.032893,0.026767
4,chr11,77997697,G,A,G,derived,chr11_77070336,0.002093,0.025199,0.023994


In [276]:
ppn_ptr_divergent_window_3d_modifying_variants_dedup = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos','ref','alt','ancestral_allele','alt_ancestral_derived']].drop_duplicates()

In [277]:
len(ppn_ptr_divergent_window_3d_modifying_variants_dedup)

51

Ancestral?

In [278]:
len(ppn_ptr_divergent_window_3d_modifying_variants_dedup[ppn_ptr_divergent_window_3d_modifying_variants_dedup['alt_ancestral_derived'] == 'ancestral'])

10

Derived?

In [279]:
len(ppn_ptr_divergent_window_3d_modifying_variants_dedup[ppn_ptr_divergent_window_3d_modifying_variants_dedup['alt_ancestral_derived'] == 'derived'])

41

Are there sites where neither bonobo nor chimpanzee are ancestral?

In [280]:
ppn_ptr_divergent_window_3d_modifying_variants_dedup[(ppn_ptr_divergent_window_3d_modifying_variants_dedup['ancestral_allele'].str.upper() != ppn_ptr_divergent_window_3d_modifying_variants_dedup['alt']) & (ppn_ptr_divergent_window_3d_modifying_variants_dedup['ancestral_allele'].str.upper() != ppn_ptr_divergent_window_3d_modifying_variants_dedup['ref'])]

,chr,pos,ref,alt,ancestral_allele,alt_ancestral_derived
8,chr12,38458311,C,T,A,derived
48,chr4,62682758,G,A,T,derived
58,chr5,35592012,G,A,C,derived


### Explained Divergence <a class = 'anchor' id = 'explaineddivergence'></a>

What is the fraction of divergence explained by a given variant?

In [281]:
ppn_ptr_divergent_window_maxes_dict = dict(zip(ppn_ptr_window_stats['window'], ppn_ptr_window_stats['maximum']))

In [282]:
ppn_ptr_divergent_window_3d_modifying_variants['max'] = ppn_ptr_divergent_window_3d_modifying_variants['window'].map(ppn_ptr_divergent_window_maxes_dict)

In [283]:
ppn_ptr_divergent_window_3d_modifying_variants.head(5)

,chr,pos,ref,alt,ancestral_allele,alt_ancestral_derived,window,mse,divergence,min,max
0,chr1,126940560,C,G,C,derived,chr1_126353408,0.011125,0.036647,0.029323,0.063015
1,chr1,153901388,A,T,A,derived,chr1_153616384,0.005189,0.017142,0.016506,0.032285
2,chr10,88057593,G,C,G,derived,chr10_87556096,0.018322,0.031364,0.026767,0.040717
3,chr10,88057595,G,T,G,derived,chr10_87556096,0.019119,0.032893,0.026767,0.040717
4,chr11,77997697,G,A,G,derived,chr11_77070336,0.002093,0.025199,0.023994,0.057039


In [284]:
ppn_ptr_divergent_window_3d_modifying_variants['divergence_explained'] = ppn_ptr_divergent_window_3d_modifying_variants['divergence']/ppn_ptr_divergent_window_3d_modifying_variants['max']
ppn_ptr_divergent_window_3d_modifying_variants.head(5)

,chr,pos,ref,alt,ancestral_allele,alt_ancestral_derived,window,mse,divergence,min,max,divergence_explained
0,chr1,126940560,C,G,C,derived,chr1_126353408,0.011125,0.036647,0.029323,0.063015,0.581557
1,chr1,153901388,A,T,A,derived,chr1_153616384,0.005189,0.017142,0.016506,0.032285,0.530974
2,chr10,88057593,G,C,G,derived,chr10_87556096,0.018322,0.031364,0.026767,0.040717,0.770293
3,chr10,88057595,G,T,G,derived,chr10_87556096,0.019119,0.032893,0.026767,0.040717,0.807843
4,chr11,77997697,G,A,G,derived,chr11_77070336,0.002093,0.025199,0.023994,0.057039,0.441779


Let's consider some statistics from this distribution.

In [285]:
ppn_ptr_divergent_window_3d_modifying_variants['divergence_explained'].min()

0.0854815608052588

In [286]:
ppn_ptr_divergent_window_3d_modifying_variants['divergence_explained'].mean()

0.5715557636188744

In [287]:
ppn_ptr_divergent_window_3d_modifying_variants['divergence_explained'].max()

1.1739817752739712

### CTCF <a class = 'anchor' id = 'ctcf'></a>

How often do these variants fall within a CTCF binding site. Let's use data from Schwalie et al. 2013.

In [288]:
CTCF_pbtBED = pybedtools.BedTool('/wynton/group/capra/projects/pan_3d_genome/data/annotations/panTro6_CTCF.bed')
CTCF_pbtBED.head(5)

chr1	5909	6180	6.166802	270	0	0
 chr1	32407	32758	12.266173	0	0	350
 chr1	35033	35334	7.860073	0	0	300
 chr1	58418	59129	24.39928	710	381	421
 chr1	61438	61919	28.7659185	460	411	451
 

And convert the 3d-modifying variant dataframe into a PBT BED.

In [289]:
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants[['chr','pos']].rename(columns = {'chr':'chr', 'pos':'end'})
ppn_ptr_divergent_window_3d_modifying_variants_BED['start'] = ppn_ptr_divergent_window_3d_modifying_variants_BED['end'] - 1
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants_BED[['chr','start','end']]
ppn_ptr_divergent_window_3d_modifying_variants_BED = ppn_ptr_divergent_window_3d_modifying_variants_BED.drop_duplicates()
ppn_ptr_divergent_window_3d_modifying_variants_BED.head(5)

,chr,start,end
0,chr1,126940559,126940560
1,chr1,153901387,153901388
2,chr10,88057592,88057593
3,chr10,88057594,88057595
4,chr11,77997696,77997697


In [290]:
len(ppn_ptr_divergent_window_3d_modifying_variants_BED)

51

In [291]:
ppn_ptr_divergent_window_3d_modifying_variants_pbtBED = pybedtools.BedTool().from_dataframe(ppn_ptr_divergent_window_3d_modifying_variants_BED).sort()

In [292]:
ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect = ppn_ptr_divergent_window_3d_modifying_variants_pbtBED.intersect(CTCF_pbtBED, c = True).to_dataframe(names=['chr','start','end','count'])
ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect = ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect.rename(columns = {'chr':'chr', 'start':'start', 'end':'pos', 'count':'CTCF'})
ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect = ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect[['chr','pos','CTCF']]
ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect.head(5)

,chr,pos,CTCF
0,chr1,126940560,1
1,chr1,153901388,1
2,chr10,88057593,1
3,chr10,88057595,1
4,chr11,77997697,1


In [293]:
len(ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect[ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect['CTCF'] > 0])

34

In [294]:
34/51

0.6666666666666666

Let's add this to the main 3d-modifying variants dataframe.

In [295]:
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF = pd.merge(ppn_ptr_divergent_window_3d_modifying_variants, ppn_ptr_divergent_window_3d_modifying_unique_variants_CTCF_intersect, on = ['chr','pos'], how = 'outer')
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF.head(5)

,chr,pos,ref,alt,ancestral_allele,alt_ancestral_derived,window,mse,divergence,min,max,divergence_explained,CTCF
0,chr1,126940560,C,G,C,derived,chr1_126353408,0.011125,0.036647,0.029323,0.063015,0.581557,1
1,chr1,153901388,A,T,A,derived,chr1_153616384,0.005189,0.017142,0.016506,0.032285,0.530974,1
2,chr10,88057593,G,C,G,derived,chr10_87556096,0.018322,0.031364,0.026767,0.040717,0.770293,1
3,chr10,88057595,G,T,G,derived,chr10_87556096,0.019119,0.032893,0.026767,0.040717,0.807843,1
4,chr11,77997697,G,A,G,derived,chr11_77070336,0.002093,0.025199,0.023994,0.057039,0.441779,1


In [296]:
len(ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF)

61

Let's investigate if we recover more CTCF sites by adding an interval around the variant.

In [297]:
ppn_ptr_divergent_window_3d_modifying_variants_1kb_buffer_pbtBED = ppn_ptr_divergent_window_3d_modifying_variants_pbtBED.slop(b = 10000, g = '/wynton/group/capra/projects/pan_3d_genome/data/metadata/panTro6_chr_lengths.txt')
ppn_ptr_divergent_window_3d_modifying_variants_1kb_buffer_pbtBED.head(5)

chr1	126930559	126950560
 chr1	153891387	153911388
 chr10	88047592	88067593
 chr10	88047594	88067595
 chr11	77987696	78007697
 

In [298]:
ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect = ppn_ptr_divergent_window_3d_modifying_variants_1kb_buffer_pbtBED.intersect(CTCF_pbtBED, c = True).to_dataframe(names=['chr','start','end','count'])
ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect = ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect.rename(columns = {'chr':'chr', 'start':'start', 'end':'pos', 'count':'CTCF'})
ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect = ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect[['chr','pos','CTCF']]
ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect.head(5)

,chr,pos,CTCF
0,chr1,126950560,4
1,chr1,153911388,2
2,chr10,88067593,2
3,chr10,88067595,2
4,chr11,78007697,1


In [299]:
len(ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect[ppn_ptr_divergent_window_3d_modifying_unique_variants_1kb_buffer_CTCF_intersect['CTCF'] > 0])

36

Do CTCF sites explain more 3d divergence?

In [300]:
divegence_explained_no_CTCF = ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF[ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF['CTCF'] == 0]['divergence_explained']
divegence_explained_no_CTCF.mean()

0.471638961035618

In [301]:
divegence_explained_CTCF = ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF[ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF['CTCF'] == 1]['divergence_explained']
divegence_explained_CTCF.mean()

0.6133814019095398

In [302]:
mannwhitneyu(divegence_explained_no_CTCF, divegence_explained_CTCF)

MannwhitneyuResult(statistic=207.0, pvalue=0.004532625650113199)

### Chromatin Contact Effect <a class = 'anchor' id = 'chromatincontacteffect'></a>

Now let's classify the effect of the 3d-modifying variant as resulting in decreased or increased chromatin contact where we can assess.

In [303]:
ppn_ptr_divergent_window_3d_modifying_variants_chromatin_contact_effects = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_window_3d_modifying_variant_contact_effects.txt', sep = '\t', header = 0)
ppn_ptr_divergent_window_3d_modifying_variants_chromatin_contact_effects.head(5)

,chr,pos,window,ref_map_contacts_sum,alt_map_contacts_sum,summed_contact_difference
0,chr1,126940560,chr1_126353408,-15.429675,-106.086547,-90.656872
1,chr1,153901388,chr1_153616384,-13.501551,-72.696401,-59.194849
2,chr10,88057593,chr10_87556096,-37.248298,-100.210362,-62.962064
3,chr10,88057595,chr10_87556096,-37.248298,-101.694820,-64.446522
4,chr11,77997697,chr11_77070336,41.417256,-11.054677,-52.471933


Add a column that indicates whether the summed contact difference is negative (decrease) or positive (increase).

In [304]:
ppn_ptr_divergent_window_3d_modifying_variants_chromatin_contact_effects['effect'] = pd.cut(ppn_ptr_divergent_window_3d_modifying_variants_chromatin_contact_effects['summed_contact_difference'], bins=[float('-Inf'), 0, float('Inf')], labels=['Decrease', 'Increase'])
ppn_ptr_divergent_window_3d_modifying_variants_chromatin_contact_effects.head(5)

,chr,pos,window,ref_map_contacts_sum,alt_map_contacts_sum,summed_contact_difference,effect
0,chr1,126940560,chr1_126353408,-15.429675,-106.086547,-90.656872,Decrease
1,chr1,153901388,chr1_153616384,-13.501551,-72.696401,-59.194849,Decrease
2,chr10,88057593,chr10_87556096,-37.248298,-100.210362,-62.962064,Decrease
3,chr10,88057595,chr10_87556096,-37.248298,-101.694820,-64.446522,Decrease
4,chr11,77997697,chr11_77070336,41.417256,-11.054677,-52.471933,Decrease


Let's map this onto the 3d-modifying variant dataframe.

In [305]:
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects = pd.merge(ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF, ppn_ptr_divergent_window_3d_modifying_variants_chromatin_contact_effects, on = ['chr','pos','window'])
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects.head(5)

,chr,pos,ref,alt,ancestral_allele,alt_ancestral_derived,window,mse,divergence,min,max,divergence_explained,CTCF,ref_map_contacts_sum,alt_map_contacts_sum,summed_contact_difference,effect
0,chr1,126940560,C,G,C,derived,chr1_126353408,0.011125,0.036647,0.029323,0.063015,0.581557,1,-15.429675,-106.086547,-90.656872,Decrease
1,chr1,153901388,A,T,A,derived,chr1_153616384,0.005189,0.017142,0.016506,0.032285,0.530974,1,-13.501551,-72.696401,-59.194849,Decrease
2,chr10,88057593,G,C,G,derived,chr10_87556096,0.018322,0.031364,0.026767,0.040717,0.770293,1,-37.248298,-100.210362,-62.962064,Decrease
3,chr10,88057595,G,T,G,derived,chr10_87556096,0.019119,0.032893,0.026767,0.040717,0.807843,1,-37.248298,-101.694820,-64.446522,Decrease
4,chr11,77997697,G,A,G,derived,chr11_77070336,0.002093,0.025199,0.023994,0.057039,0.441779,1,41.417256,-11.054677,-52.471933,Decrease


In [306]:
len(ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects)

61

In [307]:
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects.groupby(['effect']).size().to_frame('N')

,N
effect,
Decrease,38
Increase,20


Let's also stratify the effect of chromatin contact by CTCF overlap.

In [308]:
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects.groupby(['effect','CTCF']).size().to_frame('N')

N
effect   CTCF    
Decrease 0      4
         1     34
Increase 0     12
         1      8

Let's also stratify by ancestral vs. derived after excluding variants in overlapping windows that disagree in overall effect.

In [309]:
temp = ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects[['chr','pos','ref','alt','alt_ancestral_derived','CTCF','effect']].drop_duplicates().reset_index().drop([11,12,28,29,40,41])
len(temp)

48

In [310]:
temp.groupby(['effect','alt_ancestral_derived']).size().to_frame('N')

N
effect   alt_ancestral_derived    
Decrease ancestral               3
         derived                27
Increase ancestral               7
         derived                 9

Finally, we will test the summed contact distribution between variants that decrease and increase contact overall.

In [311]:
decrease_contact_3d_modifying_variants = ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects[ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects['effect'] == 'Decrease']['summed_contact_difference']
increase_contact_3d_modifying_variants = ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects[ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects['effect'] == 'Increase']['summed_contact_difference']

In [312]:
abs(decrease_contact_3d_modifying_variants).mean()

41.72285694987095

In [313]:
abs(increase_contact_3d_modifying_variants).mean()

30.289600045587456

In [314]:
mannwhitneyu(abs(decrease_contact_3d_modifying_variants), abs(increase_contact_3d_modifying_variants))

MannwhitneyuResult(statistic=483.0, pvalue=0.09358127967826899)

Save this dataframe.

In [315]:
ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ppn_ptr_windows/ppn_ptr_divergent_window_3d_modifying_variants_with_CTCF_and_effects.txt', sep = '\t', header = True, index = False)

## *In Silico* Mutagenesis for Chimpanzee Lineage-Specific Variants <a class = 'anchor' id = 'ptrinsilicomutagenesis'></a>

Let's also consider *in silico* mutagenesis run on chimpanzee lineage-specific variants. We will search for any variants with a divergence > 0.001.

In [316]:
pte_specific_variants = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/in_silico_mutagenesis/pte_specific_variants_in_silico_mutagenesis.txt', sep = '\t', header = 0)
pte_specific_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence
0,chr1,1816164,G,A,chr1_1048576,1.237785e-08,6.420484e-08
1,chr1,1816164,G,A,chr1_1572864,1.241441e-08,3.121775e-08
2,chr1,1850356,C,A,chr1_1048576,1.183865e-07,3.686226e-07
3,chr1,1850356,C,A,chr1_1572864,9.813220e-08,1.407815e-07
4,chr1,1966708,C,T,chr1_1048576,9.442934e-09,3.243256e-08


In [317]:
len(pte_specific_variants)

64657

In [318]:
len(pte_specific_variants[['chr','pos','ref','alt']].drop_duplicates())

34474

In [319]:
pte_specific_variants[pte_specific_variants['divergence'] > 0.001]

,chr,pos,ref,alt,window,mse,divergence
18725,chr14,27334853,A,C,chr14_26738688,0.000018,0.001643
28012,chr19,28725598,C,T,chr19_28311552,0.000252,0.001637
41141,chr3,155608581,G,A,chr3_155189248,0.000384,0.001303
51875,chr6,78105433,A,G,chr6_77594624,0.000537,0.007160
59002,chr8,44129911,T,C,chr8_43515904,0.001514,0.008608
63191,chr9,52984573,A,G,chr9_52428800,0.000042,0.001082


In [320]:
pts_specific_variants = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/in_silico_mutagenesis/pts_specific_variants_in_silico_mutagenesis.txt', sep = '\t', header = 0)
pts_specific_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence
0,chr1,13689020,A,G,chr1_13631488,1.512008e-07,4.080969e-07
1,chr1,13689020,A,G,chr1_13107200,7.498010e-07,1.060541e-06
2,chr1,21490047,T,C,chr1_20971520,1.515650e-09,2.777475e-09
3,chr1,21490047,T,C,chr1_20447232,1.810662e-11,1.642873e-10
4,chr1,39399758,G,A,chr1_38797312,7.904467e-08,2.444597e-07


In [321]:
len(pts_specific_variants)

610

In [322]:
len(pts_specific_variants[['chr','pos','ref','alt']].drop_duplicates())

337

In [323]:
pts_specific_variants[pts_specific_variants['divergence'] > 0.001]

,chr,pos,ref,alt,window,mse,divergence


In [324]:
ptt_specific_variants = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/in_silico_mutagenesis/ptt_specific_variants_in_silico_mutagenesis.txt', sep = '\t', header = 0)
ptt_specific_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence
0,chr1,20843059,G,T,chr1_20447232,1.436774e-08,2.963894e-08
1,chr1,36299509,T,C,chr1_35651584,3.652465e-07,8.120434e-07
2,chr1,36299509,T,C,chr1_36175872,4.638181e-07,1.475675e-06
3,chr1,51965780,G,T,chr1_51380224,8.828241e-09,1.742867e-08
4,chr1,51965780,G,T,chr1_51904512,1.869705e-09,4.626260e-09


In [325]:
len(ptt_specific_variants)

150

In [326]:
len(ptt_specific_variants[['chr','pos','ref','alt']].drop_duplicates())

78

In [327]:
ptt_specific_variants[ptt_specific_variants['divergence'] > 0.001]

,chr,pos,ref,alt,window,mse,divergence


In [328]:
ptv_specific_variants = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/in_silico_mutagenesis/ptv_specific_variants_in_silico_mutagenesis.txt', sep = '\t', header = 0)
ptv_specific_variants.head(5)

,chr,pos,ref,alt,window,mse,divergence
0,chr1,2353967,C,G,chr1_1572864,4.399900e-06,8.923979e-06
1,chr1,2353967,C,G,chr1_2097152,2.604769e-06,4.205610e-06
2,chr1,2354245,A,C,chr1_1572864,6.150447e-07,1.166121e-06
3,chr1,2354245,A,C,chr1_2097152,4.088708e-07,5.174435e-07
4,chr1,2367054,G,A,chr1_1572864,9.960170e-08,2.043531e-07


In [329]:
len(ptv_specific_variants)

22671

In [330]:
len(ptv_specific_variants[['chr','pos','ref','alt']].drop_duplicates())

11993

In [331]:
ptv_specific_variants[ptv_specific_variants['divergence'] > 0.001]

,chr,pos,ref,alt,window,mse,divergence
2482,chr10,76470135,T,C,chr10_76021760,0.000448,0.002432
9236,chr18,60956003,A,G,chr18_60293120,0.000019,0.001207
11320,chr2A,55039344,C,T,chr2A_54001664,0.000417,0.001204
11321,chr2A,55039344,C,T,chr2A_54525952,0.019976,0.079208
16860,chr5,45286251,T,C,chr5_44564480,0.005497,0.006336
16861,chr5,45286251,T,C,chr5_45088768,0.003198,0.009205


Let's merge and save the resulting dataframe for any chimpanzee subspecies-specific 3d-modifying variants.

In [332]:
pte_3d_modifying_variants = pte_specific_variants[pte_specific_variants['divergence'] > 0.001]
pte_3d_modifying_variants['lineage'] = 'pte'
ptv_3d_modifying_variants = ptv_specific_variants[ptv_specific_variants['divergence'] > 0.001]
ptv_3d_modifying_variants['lineage'] = 'ptv'
ptr_subspecies_specific_3d_modifying_variants = pd.concat([pte_3d_modifying_variants, ptv_3d_modifying_variants])
ptr_subspecies_specific_3d_modifying_variants

/scratch/cbrand/ipykernel_2007303/3631605322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pte_3d_modifying_variants['lineage'] = 'pte'
/scratch/cbrand/ipykernel_2007303/3631605322.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptv_3d_modifying_variants['lineage'] = 'ptv'


,chr,pos,ref,alt,window,mse,divergence,lineage
18725,chr14,27334853,A,C,chr14_26738688,0.000018,0.001643,pte
28012,chr19,28725598,C,T,chr19_28311552,0.000252,0.001637,pte
41141,chr3,155608581,G,A,chr3_155189248,0.000384,0.001303,pte
51875,chr6,78105433,A,G,chr6_77594624,0.000537,0.007160,pte
59002,chr8,44129911,T,C,chr8_43515904,0.001514,0.008608,pte
63191,chr9,52984573,A,G,chr9_52428800,0.000042,0.001082,pte
2482,chr10,76470135,T,C,chr10_76021760,0.000448,0.002432,ptv
9236,chr18,60956003,A,G,chr18_60293120,0.000019,0.001207,ptv
11320,chr2A,55039344,C,T,chr2A_54001664,0.000417,0.001204,ptv
11321,chr2A,55039344,C,T,chr2A_54525952,0.019976,0.079208,ptv


In [333]:
ptr_subspecies_specific_3d_modifying_variants.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/ptr_subspecies_specific_variants/ptr_subspecies_specific_3d_modifying_variants.txt', sep = '\t', header = True, index = False)

## Bonobo-Chimpanzee Gene Expression Differences <a class = 'anchor' id = 'geneexpression'></a>

Let's examine differences in gene expression between bonobos and chimpanzees using data from Brawand et al. 2011.

In [334]:
tissue_dict = dict({'SRR306811': 'prefrontal_cortex',
                 'SRR306817': 'cerebellum',
                 'SRR306818': 'cerebellum', 
                 'SRR306819': 'heart', 
                 'SRR306820': 'heart', 
                 'SRR306821': 'kidney', 
                 'SRR306822': 'kidney', 
                 'SRR306823': 'liver', 
                 'SRR306824': 'liver', 
                 'SRR306825': 'testis',
                 'SRR306827': 'prefrontal_cortex',
                 'SRR306828': 'prefrontal_cortex',
                 'SRR306829': 'cerebellum',
                 'SRR306830': 'cerebellum', 
                 'SRR306831': 'heart', 
                 'SRR306832': 'heart', 
                 'SRR306833': 'kidney', 
                 'SRR306834': 'kidney', 
                 'SRR306835': 'liver', 
                 'SRR306836': 'liver', 
                 'SRR306837': 'testis'})

Define which samples belong to which individual.

In [335]:
ppn_female = ['SRR306827','SRR306829','SRR306831','SRR306833','SRR306835']
ppn_male = ['SRR306828','SRR306830','SRR306832','SRR306834','SRR306836','SRR306837']
ptr_female = ['SRR306811','SRR306817','SRR306819','SRR306821','SRR306823']
ptr_male = ['SRR306818','SRR306820','SRR306822','SRR306824','SRR306825']

Now let's write a function to gather the read count data per individual.

In [336]:
def reads_per_individual(individual, SRR_ids_list):
    individual_read_counts_dfs_list = []
    
    for i in range(len(SRR_ids_list)):
        individual_temp_df = pd.read_csv('RNAseq/read_counts/'+SRR_ids_list[i]+'_read_counts.txt', sep = '\t', names = ['gene',SRR_ids_list[i]])
        individual_read_counts_dfs_list.append(individual_temp_df)
        
    for df in individual_read_counts_dfs_list:
        df.set_index('gene', inplace = True)
        
    individual_read_counts_df = pd.concat(individual_read_counts_dfs_list, axis = 1, sort = False).reset_index()
    
    individual_read_counts_df = individual_read_counts_df.set_index(['gene']).stack().to_frame().reset_index()
    individual_read_counts_df.rename(columns={ 'level_1': 'sample', 0: individual}, inplace = True)
    individual_read_counts_df['tissue'] = individual_read_counts_df['sample'].map(tissue_dict)
    individual_read_counts_df = individual_read_counts_df.drop('sample', axis = 1)
    individual_read_counts_df = individual_read_counts_df[~individual_read_counts_df['gene'].str.startswith('_')]
    individual_read_counts_df = individual_read_counts_df[~individual_read_counts_df['gene'].str.startswith('unassigned')]
    individual_read_counts_df = individual_read_counts_df[['gene','tissue',individual]]
    
    return individual_read_counts_df

Apply the function to all four individuals.

In [337]:
ppn_female_reads = reads_per_individual('ppn_female', ppn_female).set_index(['gene','tissue'])
ppn_male_reads = reads_per_individual('ppn_male', ppn_male).set_index(['gene','tissue'])
ptr_female_reads = reads_per_individual('ptr_female', ptr_female).set_index(['gene','tissue'])
ptr_male_reads = reads_per_individual('ptr_male', ptr_male).set_index(['gene','tissue'])

Gather those data and check out the dataframe.

In [338]:
all_read_counts = pd.concat([ppn_female_reads, ppn_male_reads, ptr_female_reads, ptr_male_reads], axis = 1, sort = False).reset_index()
all_read_counts.head(12)

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
0,A1BG,prefrontal_cortex,107.0,57,51.0,NaN
1,A1BG,cerebellum,158.0,233,84.0,136.0
2,A1BG,heart,24.0,20,2.0,4.0
3,A1BG,kidney,149.0,98,10.0,87.0
4,A1BG,liver,40254.0,14391,31371.0,56937.0
5,A1CF,prefrontal_cortex,0.0,1,0.0,NaN
6,A1CF,cerebellum,0.0,1,3.0,2.0
7,A1CF,heart,0.0,1,0.0,1.0
8,A1CF,kidney,220.0,278,435.0,1305.0
9,A1CF,liver,2929.0,1876,1151.0,2853.0


In [339]:
len(all_read_counts)

208308

Let's examine read counts for the first example bonobo-chimpanzee divergent window in Figure 3.

In [340]:
all_read_counts[all_read_counts['gene'] == 'ZNF622']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
172515,ZNF622,prefrontal_cortex,340.0,748,731.0,NaN
172516,ZNF622,cerebellum,474.0,715,1042.0,486.0
172517,ZNF622,heart,613.0,316,859.0,544.0
172518,ZNF622,kidney,352.0,335,764.0,587.0
172519,ZNF622,liver,511.0,238,1091.0,328.0
208093,ZNF622,testis,NaN,640,NaN,637.0


In [341]:
all_read_counts[all_read_counts['gene'] == 'RETREG1']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
145385,RETREG1,prefrontal_cortex,453.0,1885,398.0,NaN
145386,RETREG1,cerebellum,499.0,626,707.0,462.0
145387,RETREG1,heart,2892.0,700,1826.0,3262.0
145388,RETREG1,kidney,652.0,461,1515.0,990.0
145389,RETREG1,liver,4637.0,496,1066.0,505.0
202667,RETREG1,testis,NaN,1360,NaN,2289.0


In [342]:
all_read_counts[all_read_counts['gene'] == 'MYO10']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
129540,MYO10,prefrontal_cortex,2656.0,1943,829.0,NaN
129541,MYO10,cerebellum,609.0,833,854.0,603.0
129542,MYO10,heart,357.0,495,386.0,882.0
129543,MYO10,kidney,1914.0,1383,2184.0,3619.0
129544,MYO10,liver,288.0,427,264.0,538.0
199498,MYO10,testis,NaN,770,NaN,1918.0


In [343]:
all_read_counts[all_read_counts['gene'] == 'BASP1']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
6955,BASP1,prefrontal_cortex,6121.0,11050,9051.0,NaN
6956,BASP1,cerebellum,3102.0,2160,1754.0,1972.0
6957,BASP1,heart,114.0,62,174.0,28.0
6958,BASP1,kidney,210.0,156,213.0,189.0
6959,BASP1,liver,489.0,92,903.0,94.0
174981,BASP1,testis,NaN,36,NaN,35.0


Now for the example in Figure 4.

In [344]:
all_read_counts[all_read_counts['gene'] == 'SRI']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
155455,SRI,prefrontal_cortex,1848.0,3479,1131.0,NaN
155456,SRI,cerebellum,1309.0,1434,1359.0,713.0
155457,SRI,heart,670.0,450,409.0,467.0
155458,SRI,kidney,871.0,788,869.0,1010.0
155459,SRI,liver,316.0,194,580.0,100.0
204681,SRI,testis,NaN,260,NaN,360.0


In [345]:
all_read_counts[all_read_counts['gene'] == 'STEAP4']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
156170,STEAP4,prefrontal_cortex,12.0,44,21.0,NaN
156171,STEAP4,cerebellum,27.0,19,30.0,9.0
156172,STEAP4,heart,330.0,320,1019.0,272.0
156173,STEAP4,kidney,60.0,74,421.0,41.0
156174,STEAP4,liver,983.0,399,1951.0,289.0
204824,STEAP4,testis,NaN,108,NaN,186.0


In [346]:
all_read_counts[all_read_counts['gene'] == 'ZNF804B']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
173090,ZNF804B,prefrontal_cortex,17.0,175,23.0,NaN
173091,ZNF804B,cerebellum,47.0,78,22.0,11.0
173092,ZNF804B,heart,0.0,2,0.0,0.0
173093,ZNF804B,kidney,1.0,2,0.0,0.0
173094,ZNF804B,liver,0.0,0,0.0,0.0
208208,ZNF804B,testis,NaN,11,NaN,29.0


In [347]:
all_read_counts[all_read_counts['gene'] == 'TEX47']

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
158935,TEX47,prefrontal_cortex,1.0,3,0.0,NaN
158936,TEX47,cerebellum,9.0,10,0.0,3.0
158937,TEX47,heart,0.0,0,0.0,0.0
158938,TEX47,kidney,0.0,0,0.0,1.0
158939,TEX47,liver,0.0,2,0.0,0.0
205377,TEX47,testis,NaN,195,NaN,689.0


Now let's focus on all genes in bonobo-chimpanzee divergent windows.

In [348]:
ppn_ptr_divergent_windows_genes_read_counts = all_read_counts[all_read_counts['gene'].isin(ppn_ptr_divergent_windows_genes)]
ppn_ptr_divergent_windows_genes_read_counts = ppn_ptr_divergent_windows_genes_read_counts[(ppn_ptr_divergent_windows_genes_read_counts['ppn_female'] != 0) & (ppn_ptr_divergent_windows_genes_read_counts['ppn_male'] != 0) & (ppn_ptr_divergent_windows_genes_read_counts['ptr_female'] != 0) & (ppn_ptr_divergent_windows_genes_read_counts['ptr_male'] != 0)] 
ppn_ptr_divergent_windows_genes_read_counts = ppn_ptr_divergent_windows_genes_read_counts[~ppn_ptr_divergent_windows_genes_read_counts['tissue'].isin(['prefrontal_cortex','testis'])]
ppn_ptr_divergent_windows_genes_read_counts.head(5)

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male
511,ABL2,cerebellum,1085.0,1865,1276.0,1168.0
512,ABL2,heart,394.0,295,375.0,425.0
513,ABL2,kidney,520.0,405,516.0,649.0
514,ABL2,liver,289.0,238,266.0,163.0
531,ABO,cerebellum,2984.0,4796,2052.0,3137.0


In [349]:
len(ppn_ptr_divergent_windows_genes_read_counts)

1361

Write a function to identify genes with a species difference in expression.

In [350]:
def calculate_species_difference(dataframe):
    df = dataframe.copy()
    df['ppn_delta'] = df.apply(lambda row: abs(row['ppn_female'] - row['ppn_male']), axis=1)
    df['ptr_delta'] = df.apply(lambda row: abs(row['ptr_female'] - row['ptr_male']), axis=1)
    species_diff_list = []

    for index, row in df.iterrows():
        ppn_min = min(row['ppn_female'], row['ppn_male'])
        ppn_max = max(row['ppn_female'], row['ppn_male'])
        ptr_min = min(row['ptr_female'], row['ptr_male'])
        ptr_max = max(row['ptr_female'], row['ptr_male'])

        if ppn_min > ptr_max:
            species_diff_list.append(ppn_min - ptr_max)
        elif ptr_min > ppn_max:
            species_diff_list.append(ptr_min - ppn_max)
        else:
            species_diff_list.append(None)
        
    df['species_diff'] = species_diff_list

    return df

In [351]:
species_different_gene_expression = calculate_species_difference(ppn_ptr_divergent_windows_genes_read_counts)
species_different_gene_expression.head(5)

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male,ppn_delta,ptr_delta,species_diff
511,ABL2,cerebellum,1085.0,1865,1276.0,1168.0,780.0,108.0,NaN
512,ABL2,heart,394.0,295,375.0,425.0,99.0,50.0,NaN
513,ABL2,kidney,520.0,405,516.0,649.0,115.0,133.0,NaN
514,ABL2,liver,289.0,238,266.0,163.0,51.0,103.0,NaN
531,ABO,cerebellum,2984.0,4796,2052.0,3137.0,1812.0,1085.0,NaN


In [352]:
len(species_different_gene_expression)

1361

In [353]:
len(species_different_gene_expression[species_different_gene_expression['species_diff'].notna()])

442

In [354]:
species_different_gene_expression[species_different_gene_expression['species_diff'].notna()]

,gene,tissue,ppn_female,ppn_male,ptr_female,ptr_male,ppn_delta,ptr_delta,species_diff
1282,ADAM22,heart,31.0,47,53.0,56.0,16.0,3.0,6.0
1283,ADAM22,kidney,198.0,181,199.0,505.0,17.0,306.0,1.0
1284,ADAM22,liver,13.0,129,10.0,11.0,116.0,1.0,2.0
1437,ADAMTSL2,heart,151.0,714,89.0,55.0,563.0,34.0,62.0
1439,ADAMTSL2,liver,352.0,305,689.0,402.0,47.0,287.0,50.0
2119,AGMO,liver,254.0,221,22.0,182.0,33.0,160.0,39.0
2163,AGPAT5,kidney,187.0,199,317.0,687.0,12.0,370.0,118.0
2413,AK8,kidney,30.0,20,127.0,134.0,10.0,7.0,97.0
2471,AKAP5,cerebellum,181.0,141,64.0,102.0,40.0,38.0,39.0
2474,AKAP5,liver,37.0,55,7.0,21.0,18.0,14.0,16.0


## Cell Type Comparison <a class = 'anchor' id = 'celltypecomparison'></a>

Are the cell type specific predictions variable across cell types for the reference sequence?

In [355]:
#reference_comparisons_header = ['cell_type_1','cell_type_2','chr','window_start','mse','spearman']
#reference_comparisons = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/comparisons/reference/all_reference_comparisons.txt', sep = '\t', names = reference_comparisons_header)
#reference_comparisons['window'] = reference_comparisons['chr'] + '_' + reference_comparisons['window_start'].astype(str)
#reference_comparisons = reference_comparisons[['cell_type_1','cell_type_2','chr','window_start','window','mse','spearman']]
#reference_comparisons.head()

In [356]:
#excluded_header = ['chr','start','end','N_missing']
#excluded = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/metadata/panTro6_excluded_windows.bed', sep = '\t', header = None, names = excluded_header)
#excluded['window'] = excluded['chr'] + '_' + excluded['start'].astype(str)
#excluded_windows = excluded['window'].tolist()

In [357]:
#reference_comparisons = reference_comparisons[~(reference_comparisons['window'].isin(excluded_windows))]
#reference_comparisons.head()

Let's save this dataframe for plotting.

In [358]:
#reference_comparisons.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/dataframes/reference_cell_type_comparisons.txt', sep = '\t', header = True, index = False)

In [359]:
#len(reference_comparisons)

In [360]:
#reference_comparisons.groupby(['cell_type_1','cell_type_2'])['spearman'].mean().to_frame('Mean Rho')

Most of the maps for the different cell types are nearly identical when predicting the reference sequence.

Let's compare sample predictions from HFF to GM12878. Load the data and run a correlation.

In [361]:
#GM12878_comparisons = pd.read_csv('/wynton/group/capra/projects/pan_3d_genome/data/dataframes/GM12878_comparisons.txt', sep = '\t', header = 0)
#GM12878_comparisons.head(5)

In [362]:
#rho, p = spearmanr(comparisons['divergence'], GM12878_comparisons['divergence'])
#print(rho, p)

In [363]:
#HFF_divergence = comparisons[['ind1','ind2','window','divergence']].copy()
#HFF_divergence['cell_type'] = 'HFF'
#GM12878_divergence = GM12878_comparisons[['ind1','ind2','window','divergence']].copy()
#GM12878_divergence['cell_type'] = 'GM12878'
#cell_type_correlation = HFF_divergence.merge(GM12878_divergence, how = 'outer', on = ['ind1','ind2','window'])
#cell_type_correlation = cell_type_correlation[['ind1','ind2','window','cell_type_x','divergence_x','cell_type_y','divergence_y']]
#cell_type_correlation.head()

In [364]:
#len(cell_type_correlation)

Save this dataframe for plotting.

In [365]:
#cell_type_correlation.to_csv('/wynton/group/capra/projects/pan_3d_genome/data/dataframes/cell_type_correlation.txt', sep = '\t', header = True, index = False)

After plotting, there is a noticeable cluster of windows with low HFF divergence but high GM12878 divergence. Let's take a look.

In [366]:
#high_GM12878_windows = cell_type_correlation[(cell_type_correlation['divergence_x'] < 0.075) & (cell_type_correlation['divergence_y'] > 0.1)]
#high_GM12878_windows.head(200)

In [367]:
#len(high_GM12878_windows)

In [368]:
#high_GM12878_windows.groupby('window')['divergence_y'].count().to_frame()

Two windows stick out in particular. Are these maps very different in the reference sequence?

In [369]:
#reference_comparisons[(reference_comparisons['cell_type_1'] == 'GM12878') & (reference_comparisons['cell_type_2'] == 'HFF') & (reference_comparisons['window'] == 'chr4_94371840')]

In [370]:
#reference_comparisons[(reference_comparisons['cell_type_1'] == 'GM12878') & (reference_comparisons['cell_type_2'] == 'HFF') & (reference_comparisons['window'] == 'chr11_23068672')]